<a href="https://colab.research.google.com/github/suzukimain/diffusers_in_Colab/blob/main/diffusers_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title   (option)ドライブに接続 { run: "auto", display-mode: "form"}
Google_driveに接続 = True  # @param {type:"boolean"}

conect_drive=False

from google.colab import drive
if Google_driveに接続:
    Gdrive="GoogleDrive: \033[32m接続成功\033[0m"
    if not drive._os.path.ismount('/content/drive'):
        try:
            drive.mount('/content/drive')
            conect_drive=True
        except:
            Gdrive="GoogleDrive: \033[31m接続失敗\033[0m"
            conect_drive=False
else:
    Gdrive="GoogleDrive: \033[33m接続なし\033[0m"
    if drive._os.path.ismount('/content/drive'):
        drive.flush_and_unmount()
        conect_drive=False
        print("GoogleDriveの接続を解除しました")

In [ ]:
#@title   #Step.1 セットアップ (Setup) {display-mode: "form"}
print("実行中…")
#if torch.cuda.device_count()==False:
#  raise RuntimeError("deviceをGPUに変更お願いします")

開発版のライブラリを使用 = True  #@param {type:"boolean"}
#@markdown ※バグが起きる可能性あり

import torch,os
from importlib import reload
from google.colab import drive
import locale #この2行は非常に重要
locale.getpreferredencoding = lambda: "UTF-8"


if not drive._os.path.ismount('/content/drive'):
    conect_drive=False
else:
    conect_drive=True


if 開発版のライブラリを使用:
    !pip install --upgrade -q git+https://github.com/huggingface/diffusers.git

else:
    !pip install --upgrade -q diffusers

import diffusers
reload(diffusers)
!pip install --upgrade -q transformers omegaconf  sacremoses googletrans==3.1.0a0

import torch
import transformers
import safetensors
import huggingface_hub
import omegaconf
import glob
import gc
import time
import sys
import requests
import urllib.request
import codecs
import re
import pickle
import torch
import random
import numpy as np
import googletrans
import ipywidgets as widgets
from PIL import Image,PngImagePlugin
from requests import HTTPError
import difflib
import importlib
import inspect
import datetime
from IPython.display import display, Markdown
from diffusers import (StableDiffusionPipeline,
                       StableDiffusionImg2ImgPipeline,
                       StableDiffusionDepth2ImgPipeline,
                       DPMSolverMultistepScheduler,
                       AutoencoderKL,
                       PNDMScheduler,
                       schedulers,
                       TextToVideoZeroPipeline,
                       StableDiffusionInpaintPipeline,
                       StableDiffusionPipelineSafe,
                       DDPMScheduler,
                       DDIMScheduler,
                       PNDMScheduler,
                       LMSDiscreteScheduler,
                       EulerDiscreteScheduler,
                       EulerAncestralDiscreteScheduler,
                       DPMSolverMultistepScheduler,
                       DEISMultistepScheduler,
                       UniPCMultistepScheduler,
                       KDPM2DiscreteScheduler,
                       DPMSolverSinglestepScheduler,
                       KDPM2AncestralDiscreteScheduler,
                       HeunDiscreteScheduler,
                       )

from transformers import (pipeline,
                          CLIPTextModel,
                          CLIPTokenizer,
                          AutoModel,
                          AutoTokenizer,
                          AutoTokenizer,
                          AutoModelForCausalLM)
from diffusers.pipelines.stable_diffusion_safe import SafetyConfig
from googletrans import Translator
from IPython.display import display, HTML
from huggingface_hub import hf_hub_download
from torch import Generator
import imageio
from base64 import b64encode
from google.colab import drive
import google.colab.drive as drive
from multiprocessing import Process, Manager
import multiprocessing as mp


from diffusers import logging as df_logging
from transformers import logging as tf_logging
import threading
df_logging.set_verbosity_error()
tf_logging.set_verbosity_error()

import warnings
warnings.filterwarnings("ignore")

strength = 1
入力する画像 = ""
video_length= 8
video_fps = 8
safety_level = "MAX"
momentum = 1
momentum_hist = -0.1
history_d = "rand_init"


device="cuda" if torch.cuda.is_available() else 'cpu'

download_list=[]


def key_check(keyword) -> bool:
    global key_dict
    if "key_dict" not in globals():
        key_dict = {}
    key = str(keyword)
    key_in = False
    if key in key_dict:
        if keyword == key_dict[key]:
          key_in = True
    key_dict[key] = keyword
    return key_in

def check_url(url) -> bool:
    flag = True
    try:
        f = urllib.request.urlopen(url)
        f.close()
    except:
        flag = False
    return flag



step1_finish=True

if conect_drive:
    Connect_Gdrive="\033[34mGoogleDrive: 接続成功\033[0m"
else:
    Connect_Gdrive="\033[34mGoogleDrive: \033[33m接続なし\033[0m"
print("\n\033[34m___________________________________________\033[0m\n")
print(Connect_Gdrive)
print("\n\033[32mセットアップが完了しました。\033[0m")

In [ ]:
#@title  #Step.2 モデル選択 (Model Selection){display-mode: "form"}

#"\033[31m先にStep.1を実行してください\033[0m"


# @markdown >使用するモデルの切り替え (model change)

model_select = "Counterfeit-V2.5(Anime)(better)" # @param ["stable diffusion-v2.1(basic)", "Counterfeit-V2.5(Anime)(better)", "loliDiffusion(Anime)", "waifu diffusion-v1.4(Anime)", "Anything-v3.0(Anime)", "Anything-v4.5(Anime)", "anything-midjourney-v-4-1(Anime)", "ACertainThing(Anime)", "anime-kawai-diffusion(Anime)", "AB4.5_AC0.2(Anime)", "basil_mix(Anime)", "Counterfeit(Anime)", "Counterfeit-V2.0(Anime)", "chilled_remix(Anime)", "Double-Exposure-Diffusion(Anime)", "EimisAnimeDiffusion_1.0v(Anime)", "7th_Layer(Anime)", "Riga_Collection(Anime)", "Waifu-Diffusers(Anime)", "JWST-Deep-Space-diffusion(space)", "sd-db-epic-space-machine(space_ship)", "spacemidj(space)", "nasa-space-v2(space)", "openjourney-v4(Reality)", "Realistic_Vision_V2.0(Reality)", "meinamix_meinaV10(Reality)", "search"] {allow-input: true}
del_word_list=["(basic)","(Anime)","(Reality)","(space_ship)","(space)","(better)"]
if model_select in ["stable diffusion-v2.1(basic)", "Counterfeit-V2.5(Anime)(better)", "loliDiffusion(Anime)", "waifu diffusion-v1.4(Anime)", "Anything-v3.0(Anime)", "Anything-v4.5(Anime)", "anything-midjourney-v-4-1(Anime)", "ACertainThing(Anime)", "anime-kawai-diffusion(Anime)", "AB4.5_AC0.2(Anime)", "basil_mix(Anime)", "Counterfeit(Anime)", "Counterfeit-V2.0(Anime)", "chilled_remix(Anime)", "Double-Exposure-Diffusion(Anime)", "EimisAnimeDiffusion_1.0v(Anime)", "7th_Layer(Anime)", "Riga_Collection(Anime)", "Waifu-Diffusers(Anime)", "JWST-Deep-Space-diffusion(space)", "sd-db-epic-space-machine(space_ship)", "spacemidj(space)", "nasa-space-v2(space)", "openjourney-v4(Reality)", "Realistic_Vision_V2.0(Reality)", "meinamix_meinaV10(Reality)"]:
    for del_word in del_word_list:
        model_select=model_select.replace(del_word, "" )
#@markdown モデルの選択方法

#@markdown * ボックスの右の三角を押すと出るリストから選択

#@markdown * URLを入力

#@markdown * モデル名を入力

#@markdown * モデルが保存されているパスを入力

#@markdown *  **search** と入力すると全てのディレクトリから候補を探します。

## @markdown >モードの切り替え (mode change)
#mode_select = "fp16" #@param ["fp32","fp16"]



#hub_select = "hugface" #@param ["hugface","civitai"]


#@markdown >追加機能

auto = False  # @param {type:"boolean"}
# @markdown リポジトリ & モデルファイル を自動で選択します(推奨: ON)



if "step1_finish" not in locals():
    raise NameError("\033[33m先にStep.1の実行をお願いします\033[0m")

if "count_number" not in locals():
    count_number=0
    input_skip=False

if "model_url" not in locals():
    model_url=""

if ("token_list" not in locals()) and ("token_list" not in globals()):
    token_list=[]



class version_sort:
    def extract_version(self, version_string) -> str:
        match = re.search(r'[vV\-_](0?\d+(\.\d+)*)', version_string)
        if match:
            return match.group(1)
        else:
            return "0"

    def convert_version(self, version: str) -> int:
        converted_version = 0
        version_parts = version.split('.')
        if len(version_parts) > 0:
            for i in range(len(version_parts)):
                if i == len(version_parts) - 1:
                    converted_version += int(version_parts[i])
                else:
                    converted_version += int(version_parts[i]) * (100 ** (len(version_parts) - i - 1))
        return converted_version

    def sort(self,mylist) -> list:
        version_dict = {}
        for item in mylist:
            version = self.extract_version(item)
            converted_version = self.convert_version(version)
            version_dict[item] = converted_version
        sorted_list = sorted(version_dict, key=lambda x: version_dict[x], reverse=True)
        return sorted_list

#version=version_sort()

exts =  [".safetensors", ".ckpt"]

exclude =  ["safety_checker/model.safetensors",
            "unet/diffusion_pytorch_model.safetensors",
            "vae/diffusion_pytorch_model.safetensors",
            "text_encoder/model.safetensors",
            "unet/diffusion_pytorch_model.fp16.safetensors",
            "text_encoder/model.fp16.safetensors",
            "vae/diffusion_pytorch_model.fp16.safetensors",
            "safety_checker/model.fp16.safetensors",

            "safety_checker/model.ckpt",
            "unet/diffusion_pytorch_model.ckpt",
            "vae/diffusion_pytorch_model.ckpt",
            "text_encoder/model.ckpt",
            "text_encoder/model.fp16.ckpt",
            "safety_checker/model.fp16.ckpt",
            "unet/diffusion_pytorch_model.fp16.ckpt",
            "vae/diffusion_pytorch_model.fp16.ckpt",


                 ]


def input_fomat(self,Value,zero_txt="", return_txt="",print_txt="パス",auto=False):
        if zero_txt and return_txt:
            special_state=True
            num=0
        else:
            special_state=False
            num=1

        if type(Value) is dict:
            Value_is_dict=True
        else:
            Value_is_dict=False

        if auto:
            if Value_is_dict:
                max_pair = max(Value.items(), key=lambda x: x[1][1])
                return max_pair[1][0]
            else:
                return Value[0]

        if len(Value)>=15:
            start_number="1"
            if special_state:
                start_number="0"
                print(f"\033[34m0.{zero_txt}")

            if Value_is_dict:
                sorted_with_like = sorted(Value.items(), key=lambda x: x[1][1], reverse=True)
                for i, (model_id, (model_name, like)) in enumerate(sorted_with_like, 1):
                    if i>15:
                        break
                    print(f"\033[34m{i}.{print_txt}: {model_name}, 評価: {like}")
            else:
                for i in range(1,16):#15個
                    print(f"\033[34m{i+1}.{print_txt}: {Value[i]}\033[0m")


            print(f"\033[34m16.上記の{print_txt}以外 (全ての候補を表示します)\033[0m\n")
            while True:
                choice = input(f"{print_txt}の選択({start_number}~16): ")
                try:
                    choice=int(choice)
                except ValueError:
                    print("\033[33m自然数のみ有効です\033[34m")
                    continue
                if special_state and choice==0:
                    print("\033[0m",end="")
                    return return_txt

                if choice==16: #other_file
                    break
                elif 1<=choice<=15:
                    if Value_is_dict:
                        #choice_path,Like=Value[choice-1]
                        choice_path = Value[list(Value.keys())[choice - 1]]
                    else:
                        choice_path=Value[choice-1]
                    print("\033[0m",end="")
                    return choice_path
                else:
                    print(f"\033[33m{num}~16 までの数字の入力をお願いします\033[34m")

        start_number="1"
        if special_state:
            start_number="0"
            print(f"\033[34m0.{zero_txt}")
        for i, file_name in enumerate(Value, 1):
            print(f"\033[34m{i}.{print_txt}: {file_name}")
        while True:
            choice = input(f"使用する{print_txt}の選択({start_number}~{len(Value)}): ")
            try:
                choice=int(choice)
            except ValueError:
                print("\033[33m自然数のみ有効です\033[34m")
                continue
            if special_state and choice==0:
                print("\033[0m",end="")
                return return_txt
            if 1<=choice<=len(Value):
                if Value_is_dict:
                    #choice_path=Value[choice-1]
                    choice_path = Value[list(Value.keys())[choice - 1]]
                else:
                    choice_path=Value[choice-1]
                print("\033[0m",end="")
                return choice_path
            else:
                print(f"\033[33m{num}~{len(Value)} までの数字の入力をお願いします\033[34m")
        print("\033[0m",end="")
        return choice_path



class Huggingface(version_sort):
    def __init__(self,model_select,device,hub_select,auto):
        device=device
        self.model_id, self.model_name, self.vae_name, self.model_file="","","",""
        self.input_url=False
        self.diffuser_model=False
        self.exts=""
        self.auto=auto
        self.model_select=model_select
        self.hub=hub_select
        self.file_path_dict={}
        self.special_file=""


        self.Error_M1= (
                   '''URLを読み込めませんでした。この機能がサポートしている形式は次の通りです。
                      形式:"https://huggingface.co/<repo_name>/<model_name>/blob/main/<path_to_file>"
                      例1: "https://huggingface.co/gsdf/Counterfeit-V3.0/blob/main/Counterfeit-V3.0.safetensors"
                      例2: "https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt"
                      補足:huggingfaceにあれば、diffusersのタグがついていないモデルでも、ほとんどの場合使用できます。
                    ''')
        self.Error_M2= (
            '''hugface_pathを読み込めませんでした。この機能がサポートしている形式は次の通りです。
            形式: <repo_name>/<model_name>"
            例1: "Linaqruf/anything-v3.0"
            例2: "stabilityai/stable-diffusion-2-1"

            (下のモデル名をコピー&ペースト、または、model_selectの右側の三角を押すと出てくるドロップダウンからお選びください。)

                      "stable diffusion-v2.1"
                      "waifu diffusion-v1.4"
                      "Anything-v3.0"
                      "anything-midjourney-v-4-1"
                      "Anything-v4.5"
                      "AB4.5_AC0.2"
                      "basil_mix"
                      "Waifu-Diffusers"
                      "Double-Exposure-Diffusion"
                      "openjourney-v4"
                      "ACertainThing"
                      "Counterfeit-V2.0"
                      "Counterfeit-V2.5"
                      "7th_Layer"
                      "EimisAnimeDiffusion_1.0v"
                      "Riga_Collection"
                      "anime-kawai-diffusion"
                      "Realistic_Vision_V2.0"
                      "meinamix_meinaV10"
                      "loliDiffusion"
                      ''')
        self.Error_M3=(
                  '''指定されたパスを認識できませんでした。次をお試しください
                     ・ファイルのパスが存在するかの確認
                     ・空白が混入していないかの確認(pathの最後に半角空白が意図せずつくことがあります)
                     ・"\"や、拡張子以外の"."などの特殊記号が含まれているかの確認(認識されない可能性があります)
                  ''')

        self.special_exc=["text_encoder/",
                          "unet/",
                          "vae/",]





        self.model_dict = {
            "stable diffusion-v2.1" : "stabilityai/stable-diffusion-2-1",
            "waifu diffusion-v1.4": "hakurei/waifu-diffusion",
            "Anything-v3.0": "Linaqruf/anything-v3.0",
            "anything-midjourney-v-4-1": "Joeythemonster/anything-midjourney-v-4-1",
            "Anything-v4.5": "shibal1/anything-v4.5-clone",
            "AB4.5_AC0.2": "aioe/AB4.5_AC0.2",
            "basil_mix": "nuigurumi/basil_mix",
            "Waifu-Diffusers": "Nilaier/Waifu-Diffusers",
            "Double-Exposure-Diffusion": "joachimsallstrom/Double-Exposure-Diffusion",
            "openjourney-v4": "prompthero/openjourney-v4",
            "ACertainThing": "JosephusCheung/ACertainThing",
            "Counterfeit-V2.0": "gsdf/Counterfeit-V2.0",
            "Counterfeit-V2.5": "gsdf/Counterfeit-V2.5",
            "chilled_remix":"chilled_remix",
            "chilled_reversemix":"chilled_reversemix",
            "7th_Layer": "syaimu/7th_test",
            "EimisAnimeDiffusion_1.0v": "eimiss/EimisAnimeDiffusion_1.0v",
            "JWST-Deep-Space-diffusion" : "dallinmackay/JWST-Deep-Space-diffusion",
            "Riga_Collection": "natsusakiyomi/Riga_Collection",
            "sd-db-epic-space-machine" : "rabidgremlin/sd-db-epic-space-machine",
            "spacemidj" : "Falah/spacemidj",
            "anime-kawai-diffusion": "Ojimi/anime-kawai-diffusion",
            "Realistic_Vision_V2.0": "SG161222/Realistic_Vision_V2.0",
            "nasa-space-v2" : "sd-dreambooth-library/nasa-space-v2-768",
            "meinamix_meinaV10": "namvuong96/civit_meinamix_meinaV10",
            "loliDiffusion": "JosefJilek/loliDiffusion",
            }


        self.EN='''埋め込み: 有効\n有効化の鍵: <EasyNegative>,<bad-hands>\n
                   ※埋め込みを使用する場合、「有効化の鍵」を＜＞ごとコピー＆ペーストしてください'''


    def model_safe_check(self,model_list) ->str:
        if len(model_list)>1:
           for check_model in model_list:
                match = bool(re.search(r"(?i)[-＿]sfw", check_model))
                if match:
                    return check_model
        return model_list[0]




    def diffusers_model_check(self,checked_model: str) -> bool:
        index_url=f"https://huggingface.co/{checked_model}/blob/main/model_index.json"
        return check_url(index_url)

    def data_get(self,path) -> list:
        url = f"https://huggingface.co/api/models/{path}"
        data = requests.get(url).json()
        file_value_list = []
        df_model_bool=False
        #fix error': 'Repo model <repo_id>/<model> is gated. You must be authenticated to access it.
        try:
            siblings=data["siblings"]
        except KeyError:
            return []

        for item in siblings:
            data["siblings"]
            file_path=item["rfilename"]
            #model_index.json outside the root directory is not recognized
            if file_path=="model_index.json":
                df_model_bool=True
            elif (any(file_path.endswith(ext) for ext in exts) and
                not any(file_path.endswith(ex) for ex in exclude)):
                file_value_list.append(file_path)
        #↓{df_model,file_value_list}
        self.file_path_dict.update({path:(df_model_bool,file_value_list)})
        return file_value_list

    def model_name_search(self,model_name,auto_set):
        #"auto(bool)" loads the model with the most likes in hugface
        url = f"https://huggingface.co/api/models"#?search={model_name}"
        params={"search":model_name,"sort":"likes","direction":-1}#"downloads",}
        data = requests.get(url,params=params).json()
        with_like=[]
        choice_path=""
        repo_model_list = []
        if data:
            for item in data:
                model_id,like,private_value,tag_value = item["modelId"],item["likes"],item["private"],item["tags"]
                if ("Flax" not in tag_value and
                    "TPU" not in tag_value and
                    "audio-to-audio" not in tag_value and
                    (not private_value)):
                    File_list=self.data_get(model_id)
                    if File_list:
                        repo_model_list.append(model_id)
                        with_like.append([model_id,like])
        else:
            print("hugfaceでは適合するモデルがみつかりませんでした")
            return "_hf_no_model"

        if not auto_set:
            print("\033[34m以下のモデルパスが見つかりました。")
            if len(with_like)>=15:
                print("0.civiaiを検索")
                for (i,(model_name,like)) in enumerate(with_like,1):
                    if i>15:
                        break
                    #model_name, like=sorted_with_like[i]
                    print(f"{i}.モデルパス: {model_name}, 評価: {like}")
                print("16.上記以外")
                while True:
                    try:
                        choice = int(input("使用するモデルパスの選択: "))
                    except ValueError:
                        print("\033[33m自然数のみ有効です\033[34m")
                        continue
                    if choice==0:
                        return "_hf_no_model"
                    elif choice==16:
                        break
                    elif 1<=choice<=15:
                        choice_path=repo_model_list[choice-1]
                        return choice_path
                    else:
                        print(f"1~{len(repo_model_list)} までの数字の入力をお願いします")

            print("0.civiaiを検索")
            for (s,(model_name,like)) in enumerate(with_like,1):
                print(f"{s}.モデルパス: {model_name}, 評価: {like}")
            while True:
                try:
                    choice = int(input("使用するモデルパスの選択: "))
                except ValueError:
                    print("\033[33m自然数のみ有効です\033[34m")
                    continue
                if choice==0:
                    return "_hf_no_model"
                elif 1<=choice<=len(repo_model_list):
                    choice_path=repo_model_list[choice-1]
                    return choice_path
                else:
                    print(f"\033[33m1~{len(repo_model_list)} までの数字の入力をお願いします\033[34m")
        else:
            choice_path = with_like[0][0]

            return choice_path


    def file_name_set_sub(self,model_select,file_value):
        global old_num
        if "old_num" not in globals():
            old_num= None
        elif key_check(self.model_select) and old_num is not None and old_num!="_DFmodel":
            print(f"※前回の番号: {old_num}")

        if (not file_value) and (not self.diffuser_model):
            #return "_hf_no_model"
            raise ValueError("指定されたリポジトリから使用可能なファイルが見つかりませんでした")
        elif not file_value:
            print("\033[34m通常のDiffusersモデルのみが見つかりました")
            while True:
                result=input("使用しますか?[y/n]: ")
                if result in ["y","Y"]:
                    return "_DFmodel"
                elif result in ["n","N"]:
                    sec_result=input("civiaiの検索を実行しますか？[y/n]: ")
                    if sec_result in ["y","Y"]:
                        return "_hf_no_model"
                    elif sec_result in ["n","N"]:
                        raise ValueError("該当するモデルが見つからなかった為、処理を停止しました")
                else:
                    print("[y,n]のみの入力をお願いします")


        if len(file_value)>=15:
            start_number="1"
            if self.diffuser_model:
                start_number="0"
                print("\033[34m0.通常のDiffusersモデルを使用")
            for i in range(15):
                print(f"\033[34m{i+1}.ファイル名: {file_value[i]}\033[0m")
            print("\033[34m16.上記のファイル以外 (全ての候補を表示します)\033[0m\n")
            while True:
                choice = input(f"使用するファイルの選択({start_number}~16): ")
                try:
                    choice=int(choice)
                except ValueError:
                    print("\033[33m自然数のみ有効です\033[34m")
                    continue
                if self.diffuser_model and choice==0:
                    old_num=None
                    self.input_url=False
                    print("\033[0m",end="")
                    return "_DFmodel"

                elif choice==16: #other_file
                    break
                elif 0<=choice<=len(file_value):
                    self.input_url=True
                    old_num=choice
                    choice_path=file_value[choice-1]
                    print("\033[0m",end="")
                    return choice_path
                else:
                    print(f"\033[33m1~16 までの数字の入力をお願いします\033[34m")
            print("\033[0m",end="")
            print("\n\n")

        start_number="1"
        if self.diffuser_model:
            start_number="0"
            print("\033[34m0.通常のDiffusersモデルを使用\033[0m")
        for i, file_name in enumerate(file_value, 1):
            print(f"\033[34m{i}.ファイル名: {file_name}")
        while True:
            choice = input(f"使用するファイルの選択({start_number}~{len(file_value)}): ")
            try:
                choice=int(choice)
            except ValueError:
                print("\033[33m自然数のみ有効です\033[34m")
                continue
            if self.diffuser_model and choice==0:
                self.input_url=False
                old_num=None
                print("\033[0m",end="")
                return "_DFmodel"
            if 1<=choice<=len(file_value):
                self.input_url=True
                old_num=choice
                choice_path=file_value[choice-1]
                return choice_path
            else:
                print(f"\033[33m1~{len(file_value)} までの数字の入力をお願いします\033[34m")
        print("\033[0m",end="")





    def file_name_set(self,model_select,auto,model_type="Checkpoint"):
        if self.diffusers_model_check(model_select) and model_type=="Checkpoint":
            self.diffuser_model=True
        url = f"https://huggingface.co/api/models/{model_select}"
        response = requests.get(url)
        data = response.json()
        choice_path=""
        file_value = []
        siblings = data["siblings"]
        if data:
            for item in siblings:
                fi_path=item["rfilename"]
                if (any(fi_path.endswith(ext) for ext in exts) and
                    not any(fi_path.endswith(ex) for ex in exclude)):
                    file_value.append(fi_path)
        else:
            raise ValueError("使用可能なファイルが見つかりませんでした。\n名前の確認をお試しください")
        if file_value:
            file_value=self.sort(file_value)
            if not auto:
                print("以下のモデルファイルが見つかりました。")
                choice_path=self.file_name_set_sub(model_select,file_value)
            else:
                if self.diffuser_model:
                    self.input_url=False
                else:
                    self.input_url=True
                    choice_path=self.model_safe_check(file_value)


        elif self.diffuser_model:
            print("\033[32m通常のDifusersモデルのみが見つかりました")
            return "_DFmodel"
        else:
            raise FileNotFoundError("指定されたリポジトリに使用可能なファイルが見つかりません")
        if model_type!="Checkpoint" and model_type!="_DFmodel":
            self.input_url=False
            choice_path=hf_hub_download(repo_id=model_select, filename=choice_path)
        return choice_path


class Civitai:
    def public_civiai(self,model_select,hub,auto,model_type):
        url,filename=self.civitai_download(model_select,auto,model_type)
        filename=filename.replace(" ", "_")
        Civiai_dir="/content/model/other_model"
        Civiai_save_path=os.path.join(Civiai_dir,filename)
        os.makedirs(Civiai_dir,exist_ok=True)
        self.use_civitai(url,Civiai_save_path)
        return Civiai_save_path

    def use_civitai(self,url,savename):
        if not url in download_list:
            os.chdir("/content/model/other_model")
            os.environ["path"]=url
            os.environ["save"]=savename
            !wget -q --show-progress --content-disposition $path  -O$save
            download_list.append(url)
            os.chdir("/content")
        elif url in download_list:
            print("モデルはダウンロード済です")

        else:
            if not url in download_list:
                raise HTTPError("URLが無効です")

    def civitai_download(self,query,auto_set,model_type):
        state={}
        durl_list=[]
        files_value=[]
        durl=""
        filename=""
        like=0
        choice_path=""
        choice_file=""
        params = {"query": query,"types":model_type,"sort":"Most Downloaded"}
        response = requests.get("https://civitai.com/api/v1/models", params=params)
        if response.status_code == 200:
            data = response.json()
            items = data["items"]
            if items:
                for item in items:
                    checked_files=item["modelVersions"]
                    for check in checked_files:
                        if "files" in check:
                            files_value=check["files"]
                        repo_name=item["name"]
                        stat=item["stats"]
                        likes=stat["favoriteCount"]
                        for isa in item["modelVersions"]:
                            isa_files=isa["files"]
                            if isa_files:#important
                                for ss in isa_files:
                                    if "downloadUrl" in ss and "name" in ss:
                                        durl=ss["downloadUrl"]
                                        filename=ss["name"]
                                        break
                                    else:
                                        durl,filename="",""
                        if not durl=="":
                            durl_list.append([durl,filename])
                            state.update({len(state) + 1: (repo_name,likes)})

                if state:
                    if not auto_set:
                        sorted_with_like = sorted(state.items(), key=lambda x: x[1][1], reverse=True)
                        #if not len(state)==1:
                        print("以下のモデルパスが見つかりました")
                        if len(state)>=15:
                            for i in range(15):
                                model_name,like=state[i]
                                print(f"\033[34m{i+1}.モデルパス: {model_name}, 評価: {like}")

                            print("16.上記以外")
                            while True:
                                    choice = input("使用するモデルパスの選択: ")
                                    try:
                                        choice=int(choice)
                                    except ValueError:
                                        print("\033[33m自然数のみ有効です\033[34m")
                                        continue
                                    if choice==16:
                                        break
                                    if 1<=choice<=15:
                                        choice_path,choice_file=durl_list[choice-1]
                                        return choice_path,choice_file
                                    else:
                                        print(f"\033[33m1~{len(durl_list)} までの数字の入力をお願いします\033[34m")

                        for i, (model_name, like) in enumerate(sorted_with_like, 1):
                            print(f"\033[34m{i}.モデルパス: {model_name}, 評価: {like}")
                        while True:
                                try:
                                    choice=int(input("使用するモデルパスの選択: "))
                                except ValueError:
                                    print("\033[33m自然数のみ有効です\033[34m")
                                    continue
                                if 1<=choice<=len(durl_list):
                                    choice_path,choice_file=durl_list[choice-1]
                                    return choice_path,choice_file
                                else:
                                    print(f"\033[33m1~{len(durl_list)} までの数字の入力をお願いします\033[34m")
                        #else:
                        #    self.input_url=True
                        #    choice_path,choice_file=durl_list[0]
                    else:

                        max_likes = max(state, key=lambda x: state[x][1])
                        choice_path,choice_file = durl_list[max_likes]

                else:
                    raise ValueError("モデル情報の辞書が空です")

            else:
                raise ValueError("習得できたモデル情報が空です")
        else:
           raise HTTPError("モデル情報の習得に失敗しました")
        return choice_path,choice_file



class pipeline_setup(Huggingface,Civitai):
    def __init__(self,model_select,device,hub_select,auto):
        Huggingface.__init__(self,model_select,device,hub_select,auto)
        hub_select=hub_select

    def File_search(self):
        search_path=""
        paths = []
        for root, dirs, files in os.walk("/"):
            for file in files:
                if any(file.endswith(ext) for ext in exts):
                    path = os.path.join(root, file)
                    if path not in exclude:
                        if not path.startswith("/root/.cache"):
                            paths.append(path)
        num_path=len(paths)
        if num_path == 0:
            raise FileNotFoundError("\033[33mモデルファイルが見つかりませんでした\033[0m")
        else:
            print(f"モデルファイルの候補が{num_path}個見つかりました。")
        for s, path in enumerate(paths, 1):
            print(f"{s}: {path}")
        num = int(input(f"番号を入力してください(1〜{num_path}): "))
        if 1 <= num <= len(paths):
            search_path=(paths[num-1])
            print(f"選択されたモデルファイル: {search_path}\n")
            print('設定を変更するが、上記のモデルファイルをそのまま使う場合は、"入力を飛ばす"の適用を推奨します\n')
        else:
            raise TypeError(f"\033[33m無効な文字です。有効な数字: 1〜{len(paths)}\033[0m")
        return search_path


    def model_set(self,model_select,model_type="Checkpoint",hub_select="hugface",auto=False) -> str:
        self.auto=auto
        if not model_type  in ["Checkpoint", "TextualInversion", "LORA", "Hypernetwork", "AestheticGradient", "Controlnet", "Poses"]:
            raise TypeError('Wrong argument. Valid values are "Checkpoint", "TextualInversion", "LORA", "Hypernetwork", "AestheticGradient", "Controlnet", "Poses"')
        if not hub_select in ["hugface","civitai"]:
            raise TypeError('Wrong argument. Valid values are "hugface" or "civitai"')
        self.input_url=False
        model_path,file_path="",""
        #辞書にあるリポジトリを優先的に使う処理(テスト)
        if model_select in self.model_dict and hub_select=="hugface":
            model_path_to_check=self.model_dict[model_select]
            if check_url(f"https://huggingface.co/{model_path_to_check}"):
                model_select=model_path_to_check


        if model_select=="search":
            model_path=self.File_search()
            self.input_url=True

        elif model_select.startswith("https://huggingface.co/"):
            self.input_url=True
            if not check_url(model_select):
                raise ValueError(self.Error_M1)
            else:
                model_path=model_select

        elif model_select.startswith("https://civitai.com/"):
            model_path=self.public_civiai(model_select,hub_select,auto,model_type)


        elif os.path.isfile(model_select):
            self.input_url=True
            model_path=model_select

        elif "/" in model_select and model_select.count("/") == 1:
            if auto and self.diffusers_model_check(model_select):
                self.input_url=False
                return model_select
            else:
                file_path=self.file_name_set(model_select,auto,model_type)
                if file_path=="_hf_no_model":
                    raise ValueError("モデルが見つかりませんでした")

                elif file_path=="_DFmodel":

                    self.input_url=False
                    return model_select
                else:
                    self.input_url=True
                    model_path=f"https://huggingface.co/{model_select}/blob/main/{file_path}"


        else:
            model_path=""
            model_name_path=self.model_name_search(model_select,auto)
            if not model_name_path=="_hf_no_model":
                model_path=self.file_name_set(model_name_path,auto,model_type)
            else:
                ci_model_path=self.public_civiai(model_select,self.hub,self.auto,model_type)
                self.input_url=True
                return ci_model_path

            if model_path=="_DFmodel":
                self.input_url=False
                return model_name_path
            elif self.input_url:
                model_path=f"https://huggingface.co/{model_name_path}/blob/main/{model_path}"
            else:
                raise ValueError("モデルが見つかりません")
        return model_path


    def pipe_create(self,model_path,device):
        if self.input_url:
            base_pipe = StableDiffusionPipeline.from_single_file(
                model_path,torch_dtype=torch.float16
                ).to(device)
            print(f"\033[32mモデルPath:{model_path}")
        else:
            base_pipe = StableDiffusionPipeline.from_pretrained(
                model_path,torch_dtype=torch.float16
                ).to(device)
            print(f"\033[32mモデルID: {model_path}")
        return base_pipe

hub_select="hugface"#"civitai"
pipe_set=pipeline_setup(model_select,device,hub_select,auto)


model_path=pipe_set.model_set(model_select,hub_select=hub_select,auto=auto)
base_pipe=pipe_set.pipe_create(model_path,device)

gc.collect()

print("\033[32mモデルのセットアップが終了しました\033[0m")

In [ ]:
#@title  #Step.3 パイプラインの設定 (Pipeline Setup){display-mode: "form"}

# @markdown >パイプラインの種類

pipe_type = "txt2img" # @param ["txt2img", "img2img", "Inpaint", "Depth2Img", "txt2video", "safe"] {allow-input: true}


#@markdown >スケジューラーを変更

Scheduler_select = "DPM" # @param ["DPM","DDPM","DDIM","DEISM","DPM_S","EulerA","Euler","HeunD","K_DPM2D","K_DPM2AD","LMSD","PNDM","UniPCM","CustomEulerA","CustomEuler"] {allow-input: true}
if not Scheduler_select:
    raise TypeError("Scheduler_select has not been entered")
#@markdown >vaeを選択 (vaeを交換したい時のみ入力)
vae_select = "" # @param ["waifu-diffusion", "Counterfeit-V2.5", "anything-v3.0"] {allow-input: true}

# @markdown >フィルターを調整


# @markdown **注意事項 : 変更する時は注意して下さい**

Filter_off = False  # @param {type:"boolean"}




if vae_select:
    vae_change=True
    print("vaeを変更します")
else:
    vae_change=False




if pipe_type=="safe":
    if Scheduler_select =="CustomEulerA" or Scheduler_select == "CustomEuler":
        Scheduler_select="DDIM"
        print('safe_pipelineでは、ソナー付きの"EulerA" or "Euler"を使用すると絵が崩壊するバグが発生しているので、ddimに設定しました。')
    elif pipe_type=="safe" and Scheduler_select!=("DDIM" or "PNDM"or"LMSD"):
        print('safe_pipelineの場合、Schedulerは"ddim"or"pndm"のみ使用可能です。(Schedulerをddimに設定しました)')
        Scheduler_select="DDIM"


def get_call_method(class_name:str,method_name:str='__call__') ->list:
    call_method = getattr(getattr(diffusers, class_name),method_name)
    parameters = inspect.signature(call_method).parameters
    arg_names = []
    for param in parameters.values():
        arg_names.append(param.name)
    return arg_names

def pipe_class_type(class_name):
    call_method=get_call_method(class_name,method_name='__call__')
    if "image" in call_method:
        pipeline_type="_img2img"
    elif "video_length" in call_method:
        pipeline_type="_txt2video"
    else:
        pipeline_type="_txt2img"
    return pipeline_type


def import_on_str(desired_function_or_class,module_name="",as_arg=""):
    if as_arg:
        import_name=as_arg
    else:
        import_name=desired_function_or_class
    if not module_name:
        globals()[import_name] = __import__(desired_function_or_class)
    else:
        globals()[import_name] = getattr(__import__(module_name), desired_function_or_class)

def max_temper(text, st_list):
    similarity_dict = {}
    for string in st_list:
        similarity = difflib.SequenceMatcher(None, text, string).ratio()
        similarity_dict[string] = similarity
    most_similar_string = max(similarity_dict, key=similarity_dict.get)

    return most_similar_string

def sort_list_obj(list_obj,need_txt):
    sorted_list=[]
    for module_obj in list_obj:
        if need_txt in module_obj:
            sorted_list.append(module_obj)
    return sorted_list

def make_Error_message(output_txt,list_obj,need_txt=""):
    if need_txt:
        _diffusers_module=sort_list_obj(list_obj,need_txt)
    else:
        _diffusers_module=(dir(diffusers))
    pretxt=max_temper(output_txt,_diffusers_module)
    return pretxt


def pipe_type_set(pipe_type):
    #"StableDiffusionPipeline"
    pipe_class_dict={
            "txt2img":"StableDiffusionPipeline",
            "img2img":"StableDiffusionImg2ImgPipeline",
            "txt2video":"TextToVideoZeroPipeline",
            "Inpaint":"StableDiffusionInpaintPipeline",
            "Depth2Img":"StableDiffusionDepth2ImgPipeline",
            "safe":"StableDiffusionPipelineSafe",
            }
    if pipe_type in pipe_class_dict:
        pipe_type=pipe_class_dict[pipe_type]
    try:
        import_on_str(pipe_type,module_name="diffusers")
    except AttributeError:
        #diffusers_module_sorted=sort_list_obj(dir(diffusers),"pipeline")
        #pretxt=max_temper(pipe_type,diffusers_module_sorted)
        pretxt=make_Error_message(pipe_type,dir(diffusers),need_txt="pipeline")
        raise AttributeError(f"'{pipe_type}' not found. Maybe '{pretxt}' ?")
    return pipe_type

pipe_type=pipe_type_set(pipe_type)
pipeline_class_type=pipe_class_type(pipe_type)


if key_check(pipe_type) or "loaded_Lora_list" not in globals():
    loaded_Lora_list=[]
    loaded_textual_list=[]


word_format=f"パイプラインの種類: {pipe_type}\n"


def result():
    if Filter_off == False:
        filter_level = "フィルターの強度:通常"
    else:
        filter_level = "\033[33mフィルターの強度:無効\033[0m"
    return filter_level

#laion/CLIP-ViT-H-14-laion2B-s32B-b79K

exclude=["safety_checker/model.safetensors",
        "unet/diffusion_pytorch_model.safetensors",
        "text_encoder/model.safetensors",
        "safety_checker/model.ckpt",
        "unet/diffusion_pytorch_model.ckpt",
        "text_encoder/model.ckpt",
                 ]


class vae_set(pipeline_setup):
    def __init__(self,vae_change,vae_select,base_pipe,hub_select,auto):
        self.use_input_url=False
        self.vae_change=vae_change
        self.base_pipe=base_pipe
        self.vae_model_name=vae_select
        self.vae_path=""
        super().__init__(model_select,device,hub_select,auto)

    def vae_name_set(self,vae_model_name):
        model_names=self.model_name_search(vae_model_name,True)
        vae_path=self.file_name_set(model_names,True)
        return model_names , vae_path

    def vae_load(self):
        global vae
        model_names,vae_path=self.vae_name_set(self.vae_model_name)
        if self.diffusers_model_check(model_names):
            self.use_input_url=False
            vae_path=model_names
        else:
            self.use_input_url=True
            vae_path=f"https://huggingface.co/{model_names}/blob/main/{vae_path}"
        if self.vae_change:
            if self.use_input_url:
                vae = AutoencoderKL.from_single_file(vae_path)
            else:
                try:
                    vae = AutoencoderKL.from_pretrained(vae_path)
                except:
                    vae = AutoencoderKL.from_pretrained(vae_path , subfolder="vae")

        else:
            vae= self.base_pipe.vae
        return vae,vae_path



hub_select="hugface"
vae_setup=vae_set(vae_change,vae_select,base_pipe,hub_select,auto)

if key_check(Scheduler_select) and vae_change:
    vae,vae_path=vae_setup.vae_load()

else:
    vae_path=""
    vae=base_pipe.vae



def Euler_sh_set():
    global CustomEuler,CustomEulerA
    if not os.path.exists("/content/script/Euler_mod"):
        os.makedirs("/content/script/Euler_mod",exist_ok=True)
        !git clone https://github.com/alexblattner/modified-euler-samplers-for-sonar-diffusers.git /content/script/Euler_mod
    path_1 = "/content/script/Euler_mod/EulerANew.py"
    path_2 = "/content/script/Euler_mod/EulerNew.py"
    if path_1 not in sys.path:
        sys.path.append(path_1)
        sys.path.append(path_2)
    os.chdir("/content/script/Euler_mod")
    try:
        import_on_str("EulerA",module_name="EulerANew",as_arg="CustomEulerA")
        import_on_str("Euler",module_name="EulerNew",as_arg="CustomEuler")
        #import EulerANew as CustomEulerA
        #import EulerNew as CustomEuler
        #from EulerANew import EulerA
        #from EulerNew import Euler
    except:
        !python EulerANew.py
        !python EulerNew.py
        import_on_str("EulerA",module_name="EulerANew",as_arg="CustomEulerA")
        import_on_str("Euler",module_name="EulerNew",as_arg="CustomEuler")
        #from EulerANew import EulerA
        #from EulerNew import Euler
    os.chdir("/content")
    return CustomEulerA,CustomEuler


def Scheduler_set(Scheduler_select):
    Scheduler_dict={
            "DDPM": "DDPMScheduler",
            "DDIM": "DDIMScheduler",
            "PNDM": "PNDMScheduler",
            "LMSD" : "LMSDiscreteScheduler",
            "DPM":  "DPMSolverMultistepScheduler",
            "EulerA": "EulerAncestralDiscreteScheduler",
            "Euler": "EulerDiscreteScheduler",
            "DEISM":"DEISMultistepScheduler",
            "UniPCM":"UniPCMultistepScheduler",
            "K_DPM2D":"KDPM2DiscreteScheduler",
            "DPM_S":"DPMSolverSinglestepScheduler",
            "K_DPM2AD":"KDPM2AncestralDiscreteScheduler",
            "HeunD":"HeunDiscreteScheduler",
            }
    diffusers_module=dir(diffusers)
    if Scheduler_select in Scheduler_dict:
        Scheduler_select=Scheduler_dict[Scheduler_select]

    if Scheduler_select=="CustomEulerA" or Scheduler_select=="CustomEuler":
        CustomEulerA,CustomEuler=Euler_sh_set()
    else:
        CustomEulerA,CustomEuler=None,None
        if Scheduler_select not in diffusers_module:
            #Error_txt=max_temper(Scheduler_select, diffusers_module)
            _error=make_Error_message(Scheduler_select,dir(diffusers),need_txt="Scheduler")
            raise AttributeError(f'"{Scheduler_select}" not found. Maybe "{_error}" ?')

    return Scheduler_select,CustomEulerA,CustomEuler

Scheduler_select,CustomEulerA,CustomEuler=Scheduler_set(Scheduler_select)


EN='''埋め込み: 有効\n有効化の鍵: <EasyNegative>,<bad-hands>\n
※埋め込みを使用する場合、「有効化の鍵」を＜＞ごとコピー＆ペーストしてください'''

class make_main_pipe:
    def __init__(self,device,vae,pipe_set,base_pipe,Scheduler_select):
        self.base_pipe=base_pipe
        deivce=device
        self.Scheduler_select=Scheduler_select
        self.pipe_args_setup = {}
        self.stetas={}
        self.main_name=""
        self.vae=vae

    def main_pipe_set(self):
        txt2img_pipe,img2img_pipe,txt2video_pipe,Inpaint_pipe,safe_pipe=None,None,None,None,None
        textual_inversion_dict ={
            "EasyNegativeV2.safetensors": "EasyNegative",
            "bad-hands-5.pt": "bad-hands",
            }

        #self.Scheduler_class = Scheduler_dict[self.Scheduler_select]
        if self.Scheduler_select not in ["CustomEuler","CustomEulerA"]:
            import_on_str(self.Scheduler_select,"diffusers")
            self.Scheduler_class = getattr(diffusers, self.Scheduler_select)
        else:
            self.Scheduler_class = globals().get(self.Scheduler_select)

        scheduler = self.Scheduler_class.from_config(self.base_pipe.scheduler.config)
        self.stetas["vae"]=self.vae
        self.stetas["text_encoder"]=self.base_pipe.text_encoder
        self.stetas["tokenizer"]=self.base_pipe.tokenizer
        self.stetas["unet"]=self.base_pipe.unet
        self.stetas["scheduler"]=scheduler
        self.stetas["feature_extractor"]=self.base_pipe.feature_extractor if not Filter_off else None
        stetas_check=get_call_method(class_name=pipe_type,method_name='__init__')
        if "safety_checker" in stetas_check:
            self.stetas["safety_checker"]=self.base_pipe.safety_checker if not Filter_off else None
        elif Filter_off:
            print(f'"{pipe_type}"ではsatety_checkerの指定が使用できません')


        self.pipe_class=getattr(diffusers, pipe_type)

        main_pipe =self.pipe_class(**self.stetas).to(device, torch.float16)
        if (not pipe_type=="StableDiffusionPipelineSafe") and (token_list != "EasyNegative") and (token_list != "bad-hands"):
            textual_error=False
            try:
                main_pipe.load_textual_inversion("embed/negative", weight_name="EasyNegativeV2.safetensors", token="EasyNegative")
                main_pipe.load_textual_inversion("embed/negative", weight_name="bad-hands-5.pt", token="bad-hands")
            except ValueError:
                textual_error=True
                pass
            if not textual_error:
                token_list.append("EasyNegative")
                token_list.append("bad-hands")
            else:
                if "EasyNegative" or "bad-hands" in token_list:
                    print("既に埋め込みは適用されています。")
                    EN="\033[33m埋め込み: 適用済み\033[34m"
                else:
                    EN="\033[33m埋め込み: このモデルではsd1.5系列の埋め込みは使用不可です\033[34m"
        else:
            EN="\033[33m埋め込み: このモデルでは使用不可です\033[0m"
        try:
            main_pipe.enable_vae_tiling()
        except:
            pass
        return main_pipe

MMP=make_main_pipe(device,vae,pipe_set,base_pipe,Scheduler_select)
main_pipe=MMP.main_pipe_set()


filter_level=result()


model_print=(f"model_path: {model_path}")

def k():
    global fin
    fin=True
    return fin
fin=k()
#\033[31mが赤、\033[33mが黄色、\033[34mが青、\033[32mが緑、\033[0mが白
# "\033[32m" は緑色に変更するための"ANSI Escape Code"であり、"\033[0m"はデフォルトの文字色に戻すためのコードです。
# \033[38;2;0;255;255m　水色
#\033[38;2;74;229;110m　黄緑
if Scheduler_select!="Euler+OP"and Scheduler_select!="EulerA+OP":
    print('\n\nmomentは、"Scheduler_selectが "Euler+OP"、"EulerA+OP"のいずれかでのみ有効です。\n')

print("\033[34m____________________________________________________________________________\n\n")

print(f"model_path: {model_path}"+"\n\n")

print("scheduler:  "+Scheduler_select+"\n\n")

if vae_change:
    print(f"vae_path: {vae_path}\n\n")

print(word_format+"\n")

print(filter_level+"\n\n")


print("\033[32m画像生成の準備が出来ました。次の手順に移ってください。\n")

print("(Now that you are ready to generate images, proceed to the next step.)\033[0m")



#Option

In [ ]:
#@title  (option) Load Lora{display-mode: "form"}

model_name = "gura" # @param ["stable diffusion-v2.1(basic)", "Counterfeit-V2.5(Anime)(better)", "loliDiffusion(Anime)", "waifu diffusion-v1.4(Anime)", "Anything-v3.0(Anime)", "Anything-v4.5(Anime)", "anything-midjourney-v-4-1(Anime)", "ACertainThing(Anime)", "anime-kawai-diffusion(Anime)", "AB4.5_AC0.2(Anime)", "basil_mix(Anime)", "Counterfeit(Anime)", "Counterfeit-V2.0(Anime)", "chilled_remix(Anime)", "Double-Exposure-Diffusion(Anime)", "EimisAnimeDiffusion_1.0v(Anime)", "7th_Layer(Anime)", "Riga_Collection(Anime)", "Waifu-Diffusers(Anime)", "JWST-Deep-Space-diffusion(space)", "sd-db-epic-space-machine(space_ship)", "spacemidj(space)", "nasa-space-v2(space)", "openjourney-v4(Reality)", "Realistic_Vision_V2.0(Reality)", "meinamix_meinaV10(Reality)", "search"] {allow-input: true}
del_word_list=["(basic)","(Anime)","(Reality)","(space_ship)","(space)","(better)"]
for del_word in del_word_list:
    model_name=model_name.replace(del_word, "" )
hub_select = "civitai" #@param ["hugface","civitai"]

LORA_path=pipe_set.model_set(model_name,"LORA",hub_select=hub_select,auto=False)

try:
    main_pipe.load_lora_weights(LORA_path)
except:
    ERROR_M=("""Loraのロードに失敗しました
            原因として、SDXL用のLoraをロードしようとしている可能性もあります""")
    raise ValueError(ERROR_M)

print("\033[32mLoraのロードに成功しました\033[0m")



In [ ]:
#@title   (option)埋め込みを適用 {display-mode: "form"}
#@markdown >埋め込みを適用
Repo_id_or_path = "hololive" # @param {type:"string"}
weight_name= "" # @param {type:"string"}
token = "<gura>" # @param {type:"string"}

#NEW

Loaded_from = "hugface" # @param ["hugface", "civitai", "local"]

#LORA_path=pipe_set.model_set(model_name,"TextualInversion",hub_select=hub_select,auto=False)
loaded_textual_list=[]
class textual(pipeline_setup):
    def __init__(self,pretrained_path,weight,token,main_pipe,loaded_textual_list,hub_select="hugface"):
        pretrained_path=pretrained_path
        weight=weight
        token=token
        main_pipe=main_pipe
        loaded_textual_list=loaded_textual_list
        hub_select=hub_select
        #self.special_file=""

    def load_textual(self,pretrained_path,weight,token,main_pipe,loaded_textual_list,hub_select="hugface"):
        global token_list
        if "token_list" not in globals():
            token_list=[]
        state={"pretrained_model_name_or_path":pretrained_path,
               "token":token}
        if token in token_list:
            raise ValueError("適用済みのトークンの為、別のトークンの入力をお願いします")
        elif os.path.isfile(pretrained_path):
            pass
            #state["weight_name"]=weight_name
        else:
            embed_path=pipe_set.model_set(pretrained_path,"TextualInversion",hub_select=hub_select,auto=False)
            if hub_select=="hugface":
                state["weight_name"]=self.special_file
        try:
            main_pipe.load_textual_inversion(**state)
            token_list.append(token)
        except ValueError:
            if token in token_list:
                print("すでに指定されたトークンは適用済みです")
            else:
                print("カスタム埋め込みの適用に失敗しました")
        print(f"カスタムトークン: {','.join(token_list)}\n")

text_c=textual(Repo_id_or_path,weight_name,token,main_pipe,loaded_textual_list,hub_select=Loaded_from)
#ct=text_cadd_textual(Repo_id_or_path,weight_name,token,main_pipe)
text_c.load_textual(Repo_id_or_path,weight_name,token,main_pipe,loaded_textual_list,hub_select=Loaded_from)






#main step

In [ ]:
#@title  #Step.4 画像生成  (iamge generation){display-mode: "form"}

#@markdown ># 基本設定

# @markdown >基本ステータス


Prompt = "Please draw a beautiful Mount Fuji with the sun rising from the summit" # @param ["smail,girl, white hair, medium hair, cat ears, looking at viewer, :3, cute,white_dress", "smail,1girl, {white,blue,red,purple} hair, medium hair, cat ears, looking at viewer, :3, cute,white_dress,loli", "Please draw a beautiful Mount Fuji with the sun rising from the summit", "Earth, space, high resolution"] {allow-input: true}

N_prompt = "logo,text"  # @param {type:"string"}

seed = -1 # @param {type:"number"}


num_imgs = 3 # @param {type:"number"}

guidance_scale = 7.5 # @param {type:"slider", min:5, max:15, step:0.5}


# @markdown >追加ステータス


拡散ステップ = 30  # @param {type:"number"}
縦の大きさ = "512" #@param ["480","512","600", "768","800", "1080","1152", "1440","1920", "3840","4000","7680"] {allow-input: true}
横の大きさ = "512" #@param ["480","512","600", "768","800", "1080","1152", "1440", "1920", "3840","4000","7680"] {allow-input: true}

text_generate_model = "gpt2-prompt-generator" #@param ["None","MagicPrompt-Stable-Diffusion","anime-anything-promptgen-v2","gpt2-prompt-generator"]

#@markdown ># 詳細設定

# @markdown >出力設定

グリッドの横の画像数 = 6 # @param {type:"slider", min:1, max:10, step:1}
num_grid=int(グリッドの横の画像数)

保存する先のパス = "/content/drive/MyDrive/生成した画像/No.1" # @param ["/content/drive/MyDrive/"] {allow-input: true}
保存する先のパス = 保存する先のパス.strip()

file_name = "" #@param {type:"string"}


# @markdown >追加機能
推奨設定 = True #@param {type:"boolean"}
Recommend=推奨設定
画像を表示 = True  # @param {type:"boolean"}
グリッド画像を表示 = True  # @param {type:"boolean"}
画像情報を表示 = True  # @param {type:"boolean"}
条件をメタデーターとして追加する = True  # @param {type:"boolean"}


#NP制限トークン数拡大 = True #@param {type:"boolean"}


日本語入力 = False #@param {type:"boolean"}
JP_INPUT=日本語入力


警告メッセージを非表示 = False #@param {type:"boolean"}





if (推奨設定 and
 (momentum-momentum_hist)<=0.3 and
 (Scheduler_select=="EulerA+OP" or Scheduler_select=="Euler+OP")):
        momentum_hist=momentum-0.3
        print(f"momentum_histとmomentumの差が3以下のためmomentum_histを'{momentum_hist}' に設定しました。\nこの処理は、'推奨設定'に付属している機能です")


sd_step=拡散ステップ

height = int(縦の大きさ)
width = int(横の大きさ)




a1=time.time()





if "can_EN" not in locals():
    can_EN = False

if "text_generate_model" not in locals():
    text_generate_model="None"


if (not 推奨設定) and (not 警告メッセージを非表示):
    print('\033[33m"推奨設定"がオフになっています。')



def run_html_js(path,moji):
    import datetime
    num=len(path)
    now_a = datetime.datetime.now()
    datetimes = now_a.strftime("%Y%m%d%H%M%S")
    html_dis = f'''
    <style>
      #clipborad-text-{datetimes} {{
        border: none;
        color: #0ff;
        font-size: 15px;
      }}
    </style>
    <span style="color: #4ae56e" font-size:16px>{moji}</span> <!-- <p>タグを<span>タグに変更 -->
    <input type="text" value="{path}" id="clipborad-text-{datetimes}" size="{num}" readonly> <!-- size属性とid属性を追加 -->
    <button id="copy-button-{datetimes}" onclick="copyToClipboard('{datetimes}')">Copy</button> <!-- id属性とonclick属性を追加 -->
    '''
    js_code = '''
    function copyToClipboard(datetimes) {
      var copyText = document.getElementById("clipborad-text-" + datetimes); // datetimeを結合
      var copyButton = document.getElementById("copy-button-" + datetimes); // datetimeを結合
      copyText.select();
      navigator.clipboard.writeText(copyText.value);
      document.execCommand("copy");
      copyButton.textContent = "Copied!"; // ボタンの文字を変更
      setTimeout(function() {
        copyButton.textContent = "Copy"; // 1秒後に元に戻す
        }, 1000);
    }
    '''
    display(HTML(html_dis))
    display(HTML('<script>{}</script>'.format(js_code)))


def img_set(path):
    #対策↓:AttributeError: 'str' object has no attribute 'seek'
    if os.path.isfile(path):
        init_image = Image.open(path)
        init_image = init_image.resize((height, width))
    else:
        init_image = None
        if not pipe_type=="txt2img"or"safe":
            raise FileNotFoundError('\033[31m"入力する画像"のpathが存在しません。使用しない場合は空白にお願いします\033[0m')
    return init_image


class Scheduler_setup(make_main_pipe):
    def __init__(self,main_pipe):
        self.main_pipe=main_pipe

    def moment_set(self,Scheduler_select,vae):
        if Scheduler_select=="Euler+OP" or Scheduler_select=="EulerA+OP":
            self.main_pipe.scheduler.history_d =history_d
            self.main_pipe.scheduler.momentum = momentum
            self.main_pipe.scheduler.momentum_hist = momentum_hist
sc_set=Scheduler_setup(main_pipe)
sc_set.moment_set(Scheduler_select,vae)
if (not Prompt.endswith(",")) and (Prompt is not None):
    Prompt=Prompt+","

img_dir=False
img_dir_num=1
img_path=""
img_dict = []
if pipe_type=="img2img":
    if os.path.isfile(入力する画像) and 入力する画像.endswith((".png", ".jpg", ".jpeg")):
        img_path=入力する画像
    elif os.path.isdir(入力する画像):
        files = os.listdir(入力する画像)
        img_ext = [".jpg", ".png",".jpeg"]
        for file in files:
            ext = os.path.splitext(file)[1]
            if ext in img_ext:
                img_path = os.path.join(入力する画像, file)
                img_dict.append(img_path)
        if img_dict:
            img_dir=True
            img_dir_num=len(img_dict)
        else:
            raise FileNotFoundError("指定されたフォルダ内に画像が見つかりませんでした")
    else:
        raise FileNotFoundError('入力する画像が見つかりませんでした。画像を入力しない場合は、Step.1のパイプラインの種類を"txt2img"に変更お願いします')

if not img_dir:
    num=num_imgs
else:
    num=num_imgs*(len(img_dict))

if JP_INPUT:
    tr = Translator()
    Prompt_2_a = tr.translate(Prompt, src="ja", dest="en").text
    Prompt_2=Prompt_2_a
else:
    Prompt_2=Prompt






def txt_pipe(text_generate_model):
    global nlp
    txt_pipe_dict={"MagicPrompt-Stable-Diffusion":"Gustavosta/MagicPrompt-Stable-Diffusion",
                   "anime-anything-promptgen-v2":"FredZhang7/anime-anything-promptgen-v2",
                   "gpt2-prompt-generator":"Ar4ikov/gpt2-medium-650k-stable-diffusion-prompt-generator",
                   "None":"None"}


    txt_model_name = txt_pipe_dict[text_generate_model]
    if "nlp" not in globals():
        nlp=None

    if txt_model_name!="None" and (not key_check(txt_model_name)):
        model = AutoModelForCausalLM.from_pretrained(txt_model_name)
        tokenizer = AutoTokenizer.from_pretrained(txt_model_name)
        nlp = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device, pad_token_id=50256)
    return nlp
nlp=txt_pipe(text_generate_model)




txt_generator = torch.Generator(device)



class txt_model_setup:
    def __init__(self,nlp,Prompt,Prompt_2,add_good_prompt,JP_INPUT,text_generate_model,N_prompt):
        global Prompt_list , make_images_list
        Prompt_list=[]
        make_images_list=[]
        self.text_model_name=None
        self.pipe_name=None
        Prompt=Prompt
        self.N_prompt=N_prompt
        Prompt_2=Prompt_2
        self.add_good_prompt=推奨設定
        self.tokens=None
        JP_INPUT=JP_INPUT
        self.txt_model_name=""
        self.txt_pipe_name=""
        nlp=nlp
        self.good_word = "masterpiece:2.0,best quality,high quality,"
        text_generate_model=text_generate_model
        if self.add_good_prompt:
            self.mini_prompt=True
        else:
            self.mini_prompt=False



    def get_tokens_as_list(self,word_list):
        tokens_list = []
        for word in word_list:
            tokenized_word = self.tokenizer_with_prefix_space([word], add_special_tokens=False).input_ids[0]
            tokens_list.append(tokenized_word)
        return tokens_list

    def p_token(self,Prompt):
        self.tokens = int(len(self.tokenizer.tokenize(Prompt)))
        return self.tokens
    def txt_G(self,nlp,Prompt_2):
        n_word=["EasyNegative","bat_hands","white background"," simple background","  simple background","Loss of eye highlights","simple background","Fingers fused together","Writing Sweet Fingers","bad hands","bad legs","worst quality","low quality","Not five fingers","blurred","Missing finger","Simple background","Cat with deformed face","medium quality","purple hair","Loss of eye highlights","Fingers fused together","Writing Sweet Fingers","deleted","lowres","Low quality animals","deformed animals","hands emerging from impossible places","bad anatomy","more than three limbs hands/legs","low resolution","blurry","absurdres","pixelated","sketchy","nonsensical anatomy","unrealistic pose","mosaic","unclear details","distorted colors","unrealistic proportions","poor quality","fuzzy","missing head:1.6","out of focus","hazy","grainy","text","error","missing fingers:0.9","extra digit","fewer digits","cropped","jpeg artifacts","signature","watermark","username","standard quality","bad feet_hand_finger_leg_eye","bad","text font ui","bad shadow","poorly drawn","black-white","ugly","duplicate","mutation","mutilated","malformed mutated:1.1","malformed:1.1","The background is incoherent","simple background","low-quality background","low background","bad body","long body","broken limb","anatomical nonsense","extra limbs","missing limb","incorrect limb","multiple heads","twisted head","poorly drawn face","1 unit with multiple heads:1.3","heads together:1.0","abnormal eye:1.2 proportion","cropped:1.0","bad eyes","fused eyes","poorly drawn eyes","bad mouth","poorly drawn mouth","bad tongue","too long tongue","bad ears","poorly drawn ears","extra ears","heavy ears","long neck","too thick neck","bad neck","bad breasts","missing arms","disappearing arms","extra arms","three arms:2.0","mutated hands and fingers","fused hand","missing fingers","extra digits","huge thighs","disappearing thigh","missing thighs","extra thighs","bad feet","huge calf","disappearing legs","bad gloves","fused gloves","beard","artist name","text watermark","unnatural","obviously wrong","distorted face","floating hair","floating body parts","severed body parts","incorrect leg position","deformed","fused body and hands","disregard of physics","distorted shape","doll-like object not present in the image","body fusion","abnormal fingers","fingers resembling fish fins","dot eyes","unclear background","mosaic","body bending","incorrect leg-to-torso ratio","excessively large breasts","unsettling appearance","eyes filled with solid color","lack of lower body","splitting","creepy doll-like appearance","distorted eyes","lines on the skin","legs bending in unnatural directions","abnormal finger count","missing arms","floating hands","lack of nose or mouth","incorrect body part ratios","bad","longbody","lowres","bad anatomy","bad hands","missing fingers","Distorted eye contour","Missing part from the ankles onward","extra digit","fewer digits","split wings","Vampire wings floating in the air","bad wing","wonder egg priority","egg priority","demon"]
        if not N_prompt=="":
            words = N_prompt.split(",")
            n_word.extend(words)
        if self.mini_prompt==True:
            self.max_length=64
        else:
            self.max_length=74

        Prompt_4_M = nlp(Prompt_2, max_length=self.max_length, num_return_sequences=10 ,repetition_penalty=1.2, early_stopping=False ,do_sample=True, temperature=0.4, top_k=10)
        for V in range(len(Prompt_4_M)):
            Prompt_4_M[V] = str(Prompt_4_M[V]['generated_text']).replace('  ', '').lstrip(',')
        Prompt_4_I=''.join(Prompt_4_M)
        Prompt_4_I=Prompt_4_M[0]
        af_text = Prompt_4_I.replace(",,", ",").replace(",,,", ",").replace(", ,",",")
        if self.mini_prompt==True:
            Prompt_4=self.good_word+af_text
        else:
            Prompt_4=af_text
        return Prompt_4

    def easy_prompt(self,nlp,Prompt_2,text_generate_model):
        txt_pipe_dict={"MagicPrompt-Stable-Diffusion":("Gustavosta/MagicPrompt-Stable-Diffusion","MagicPrompt_base","MagicPrompt_tokenizer","MagicPrompt_with"),
                       "anime-anything-promptgen-v2":("FredZhang7/anime-anything-promptgen-v2","AnythingPrompt_base","AnythingPrompt_tokenizer","AnythingPrompt_with"),
                       "gpt2-prompt-generator":("Ar4ikov/gpt2-medium-650k-stable-diffusion-prompt-generator","gpt2-prompt-generator_base","gpt2-prompt-generator_tokenizer","gpt2-prompt-generator_with"),
                       "None":("","","","")
                       }
        self.txt_model_name,self.model,self.tokenizer,self.tokenizer_with_prefix_space = txt_pipe_dict[text_generate_model]
        Prompt_2_G=str(Prompt_2)
        contents = re.findall("\{(.*?)\}", Prompt_2_G)
        count = 0
        for content in contents:
            if not content or len(content) == 0:
                continue
            lst = content.split(",")
            choice = random.choice(lst)
            Prompt_2_G = re.sub("\{.*?\}", choice, Prompt_2_G, count=1)
            count += 1
        if text_generate_model == "None":
            self.good_word = "masterpiece:2.0,best quality,high quality,"
            text_model_name =""
            if self.mini_prompt:
                Prompt_4=self.good_word+Prompt_2_G
            else:
                Prompt_4 = Prompt_2_G
        else:
            Prompt_4 = self.txt_G(nlp,Prompt_2_G)
        return Prompt_4


    def main_task(self,nlp,Prompt_2):
        Prompt_4=self.easy_prompt(nlp,Prompt_2,text_generate_model)
        return Prompt_4
    def sub_task(self,nlp,Prompt_2,Prompt_list,num):
        for U in range(num):
            Prompt_4=self.main_task(nlp,Prompt_2)
            Prompt_list.append(Prompt_4)


    def use_over_token(self,PROMPT,pipe):
        max_length = pipe.tokenizer.model_max_length
        input_ids = pipe.tokenizer(PROMPT, return_tensors="pt").input_ids
        input_ids = input_ids.to("cuda")

        negative_ids = pipe.tokenizer("", truncation=False, padding="max_length", max_length=input_ids.shape[-1], return_tensors="pt").input_ids
        negative_ids = negative_ids.to("cuda")

        concat_embeds = []
        neg_embeds = []
        for i in range(0, input_ids.shape[-1], max_length):
            concat_embeds.append(pipe.text_encoder(input_ids[:, i: i + max_length])[0])
            neg_embeds.append(pipe.text_encoder(negative_ids[:, i: i + max_length])[0])

        #prompt_embeds = torch.cat(concat_embeds, dim=1)
        negative_prompt_embeds = torch.cat(neg_embeds, dim=1)
        return negative_prompt_embeds


tms=txt_model_setup(nlp,Prompt,Prompt_2,Recommend,JP_INPUT,text_generate_model,N_prompt)
CPU_P_2 = threading.Thread(target=tms.sub_task,args=(nlp,Prompt_2,Prompt_list,num))
if pipe_type=="txt2video":
    Prompt_4= tms.main_task(nlp,Prompt_2)
else:
    CPU_P_2.start()
    Prompt_4=""



if 推奨設定:
    おすすめのネガティブプロント = "Loss of eye :1.5,Fingers fused together:1.3,Writing Sweet Fingers:2.0,bad hands:2.0,bad legs:2.0,EasyNegative,bat_hands:1.3,worst quality:2.0, low quality:2.0,Fused fingers,7 fingers, 6 fingers, 4 fingers, 3 fingers,Not five fingers:2.0,blurred,Simple_background:2.0,Missing finger:1.7,Cat with deformed face:1.3 ,medium quality, purple hair,Loss of eye highlights:1.5,Fingers fused together:1.3,Writing Sweet Fingers:2.0 ,deleted:0.5, lowres,Low quality animals, deformed animals ,hands emerging from impossible places:1.7, bad anatomy, more than three limbs hands/legs:1.5, low resolution, blurry, absurdres,pixelated, sketchy, nonsensical anatomy, unrealistic pose, mosaic, unclear details, distorted colors, unrealistic proportions, poor quality, fuzzy, missing head:1.6, out of focus, hazy, grainy, text, error, missing fingers:0.9, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, standard quality, bad feet_hand_finger_leg_eye, bad, text font ui, bad shadow, poorly drawn, black-white, ugly, duplicate, mutation, mutilated, malformed mutated:1.1, malformed:1.1, The background is incoherent, simple background, low-quality background, low background, bad body, long body, broken limb, anatomical nonsense, extra limbs, missing limb, incorrect limb, multiple heads, twisted head, poorly drawn face, 1 unit with multiple heads:1.3, heads together:1.0, abnormal eye:1.2 proportion, cropped:1.0, bad eyes, fused eyes, poorly drawn eyes, bad mouth, poorly drawn mouth, bad tongue, too long tongue, bad ears, poorly drawn ears, extra ears, heavy ears, long neck, too thick neck, bad neck,  bad breasts, missing arms, disappearing arms, extra arms, three arms:2.0, mutated hands and fingers, fused hand, missing fingers, extra digits, huge thighs, disappearing thigh, missing thighs, extra thighs, bad feet, huge calf, disappearing legs, bad gloves, fused gloves, beard, artist name, text watermark, unnatural, obviously wrong, distorted face, floating hair, floating body parts, severed body parts, incorrect leg position, deformed, fused body and hands, disregard of physics, distorted shape, doll-like object not present in the image, body fusion, abnormal fingers, fingers resembling fish fins, dot eyes, unclear background, mosaic, body bending, incorrect leg-to-torso ratio, excessively large breasts, unsettling appearance, eyes filled with solid color, lack of lower body, splitting, creepy doll-like appearance, distorted eyes, lines on the skin, legs bending in unnatural directions, abnormal finger count, missing arms, floating hands, lack of nose or mouth,, incorrect body part ratios, bad, longbody, lowres, bad anatomy, bad hands, missing fingers,  Distorted eye contour, Missing part from the ankles onward, extra digit, fewer digits, split wings, Vampire wings floating in the air, bad wing, comic,chainsaw man,demon, simple background"
    if N_prompt:
        おすすめのネガティブプロント=","+おすすめのネガティブプロント

else:
    おすすめのネガティブプロント =""
    if not 警告メッセージを非表示:
        print('\033[33m"推奨ネガティブプロンプト"がオフになっています。\033[0m')


negative_prompt2 = N_prompt + おすすめのネガティブプロント




def image_grid(imgs,cols): #縦よこ
    all_num=len(imgs)
    if all_num<cols:
        cols=all_num
    if all_num>1:
        am=0
        w, h = imgs[0].size
        rows,b= divmod(all_num,cols)
        if b!=0:
            rows+=1
            am=cols-b
            white_img = Image.new("RGB", (w,h), (255, 255, 255))
            for x in range(am):
                imgs.append(white_img)
        grid = Image.new('RGB', size=(cols*w, rows*h))
        grid_w, grid_h = grid.size
        for i, img in enumerate(imgs):
            grid.paste(img, box=(i%cols*w, i//cols*h))
    elif all_num==1:
        grid=None
        print("画像が1枚のためグリッド画像化できません")
    else:
        grid=None
        print("グリッド画像に使用可能な画像がありません。")
    return grid


if "<<n>>" in 保存する先のパス:
    for nu in range(1,100000):
        save_dir_path, save_file_path=os.path.split(保存する先のパス)
        _test_save_file_path = save_file_path.replace("<<n>>", str(nu))
        if not os.path.exists(_test_save_file_path):
            保存する先のパス= os.path.join(save_dir_path,_test_save_file_path)
            break

if "<<a>>" in 保存する先のパス:
    for nu in range(1,100000):
        save_dir_path, save_file_path=os.path.split(保存する先のパス)
        _test_save_file_path = save_file_path.replace("<<a>>", str(nu))
        if "alls" in globals():
            save_file_path=save_file_path.replace("<<a>>", str(alls))
            保存する先のパス= os.path.join(save_dir_path,save_file_path)
            break
        elif not os.path.exists(_test_save_file_path):
            保存する先のパス= os.path.join(save_dir_path,_test_save_file_path)
            alls=nu
            break


if not 保存する先のパス:
    保存する先のパス="/content/Generated_images"
    print("保存する先のパスが未入力のため、デフォルトのパスに保存しました。")
if not conect_drive:
    if "/content/drive/MyDrive" in 保存する先のパス:
      保存する先のパス="/content/Generated_images"
      print("\033[31mGoogleドライブに接続されていないためデフォルトのパスに保存しました\033[0m")
    else:
        if not 保存する先のパス:
            保存する先のパス="/content/Generated_images"


image_save_path=os.path.join(保存する先のパス,"Images")

Grid_save_path=os.path.join(保存する先のパス,"Grid")
os.makedirs(image_save_path, exist_ok=True)
os.makedirs(Grid_save_path, exist_ok=True)


def seed_set(seed_number):
    if seed_number==-1 or seed_number is None:
        seed = random.randint(1,1000000)
    else:
        seed=seed_number
    return seed
seed=seed_set(seed)


def path_token_video(Prompt_4,seed_number,model_name,guidance_scale,file_name,image_save_path):
    global L
    try:
        L+=1
    except:
        L=1
    file_name_old=""
    if not file_name:
        file_name=("Gvideo")
    now = datetime.datetime.now()
    file_name_encoded = codecs.encode(file_name, 'utf-8')
    file_name_decoded = codecs.decode(file_name_encoded, 'utf-8')
    path_d = {"{prompt}":Prompt_4,"{seed}": seed, "{model_name}": model_name, "{g_scale}": guidance_scale,"{time}":now ,"{number}":L}
    for key, value in path_d.items():
        file_name_old = file_name_decoded.replace(key, str(value))
    while True:
        file_new=(f"{file_name_old}-{L}.mp4")
        file_test = os.path.join(image_save_path , file_new)
        if os.path.exists(file_test):
            L+=1
        else:
           path_new=file_test
           file_new=file_new
           break
    return path_new,file_new


def path_token_img(Prompt_4,seed_number,model_name,guidance_scale,file_name,image_save_path):
    global z
    try:
        z+=1
    except:
        z=1
    file_name_old=""
    if not file_name:
        file_name=("GIMG")
    now = datetime.datetime.now()
    file_name_encoded = codecs.encode(file_name, 'utf-8')
    file_name_decoded = codecs.decode(file_name_encoded, 'utf-8')
    path_d = {"{prompt}":Prompt_4,"{seed}": seed, "{model_name}": model_name, "{g_scale}": guidance_scale,"{time}":now ,"{number}":z}
    for key, value in path_d.items():
        file_name_old = file_name_decoded.replace(key, str(value))
    while True:
        file_new=(f"{file_name_old}-{z}.png")
        file_test = os.path.join(image_save_path , file_new)
        if os.path.exists(file_test):
            z+=1
        else:
            path_new=file_test
            file_new=file_new
            break
    return path_new,file_new



def play_mp4(path_new):
    mp4 = open(path_new, 'rb').read()
    data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
    HTML_code=(f"""
               <video width="70%" height="70%" controls>
                     <source src="{data_url}" type="video/mp4">
               </video>""")
    try:
        display(HTML(HTML_code))
    except:
        display(HTML('<script>{}</script>'.format(HTML_code)))



generator = torch.Generator(device)


class generate_class:
    def __init__(self,main_pipe,Prompt_list,make_images_list,Grid_save_path,img_dir,img_path,img_dict,seed,generator,num_grid,Prompt_4,image_save_path,sd_step,model_name,file_name):
        seed=seed
        #main_pipe=main_pipe
        generator=generator
        self.num_grid=num_grid
        self.st=num
        self.Prompt_4=Prompt_4
        Prompt_4=Prompt_4
        image_save_path=image_save_path
        self.image_save_path=image_save_path
        sd_step=sd_step
        model_name=model_name
        file_name=file_name
        self.num_imgs=num_imgs
        grid_img=None
        self.img_dir=img_dir
        self.img_path=img_path
        self.img_dict=img_dict
        self.Grid_save_path=Grid_save_path
        self.init_image=None
        self.grid_imgs=[]
        self.generate_time_all_after=0
        self.nsfw_content_detected=None
        self.generate_time_all=0
        self.generate_start_time=0
        self.generate_end_time=0
        self.i=0
        self.CPU_P_do=False
        self.CPU_P=None
        self.GPU_P=None
        Prompt_list,make_images_list=Prompt_list,make_images_list
        self.generate_num=1
        self.Prompt_use=""
        self.call_method=get_call_method(pipe_type,method_name='__call__')

    def generate_images(self,Prompt_list,make_images_list,seed,generator,image_save_path,sd_step,model_name,file_name,main_pipe):
        for self.j in range(self.st):
                #生成されない時ここのインデントを確認する
                while True:
                    if len(Prompt_list)>0:
                        self.Prompt_use=Prompt_list.pop(0)
                        break
                    else:
                        time.sleep(0.5)
                        #continueがついていると、生成されない
                Prompt_use=self.Prompt_use
                path_new,file_new=path_token_img(Prompt_use,seed,model_name,guidance_scale,file_name,image_save_path)
                if seed==-1:
                    seed=seed_set(seed)
                now = datetime.datetime.now()
                date_str = now.strftime("%Y-%m-%d_UTC-%H:%M:%S")
                path = os.path.join(image_save_path, file_name)
                generate_start_time = time.time()
                generator.manual_seed(seed)

                call_dict={}
                if "image" in self.call_method:
                    call_dict["image"]=self.init_image
                    call_dict["strength"]=strength
                else:
                    call_dict["height"]=height
                    call_dict["width"]=width
                output = main_pipe(prompt=Prompt_use, negative_prompt=negative_prompt2, guidance_scale=guidance_scale, num_inference_steps=sd_step,generator=generator,**call_dict)

                self.nsfw_content_detected=output.nsfw_content_detected
                make_image=output.images[0]
                if self.nsfw_content_detected is None:
                    bloke=False
                else:
                    if False in self.nsfw_content_detected:
                        bloke=False
                    else:
                        bloke=True
                info = make_image.info
                metadata = {
                    "Seed": seed,
                    "G_scale":guidance_scale,
                    "D_step":sd_step,
                    "model_path":model_path,
                    "Prompt": Prompt_use,
                    "n_prompt":negative_prompt2,

                    }


                generate_end_time = time.time()
                self.generate_time = generate_end_time - generate_start_time
                self.generate_time_all += self.generate_time
                self.generate_time_after=("{:.2f}s".format(self.generate_time))
                #if 枚数制限なし:
                #    text_0=(f"\033[34m画像生成が完了しました ({self.generate_num}/∞)  {self.generate_time_after}")
                #else:
                text_0=(f"\033[34m画像生成が完了しました ({self.generate_num}/{self.st})  {self.generate_time_after}")
                text_1=(f"""{text_0}\nseed値:\033[38;2;0;255;255m {seed}\n\033[34mファイルの名前: \033[32m{file_new}\n\033[0m""")
                text_2=(f"\033[92mプロンプト:  {Prompt_use}\033[0m")
                make_images_list.append([make_image,path_new,bloke,metadata,text_1,text_2])
                if not bloke:
                    self.grid_imgs.append(make_image)
                else:
                    pass
                self.generate_num+=1

    def last_task(self,make_images_list):
        for T in range(self.st):
            while True:
                if len(make_images_list)>0:
                    make_image,path_new,bloke,metadata,text_1,text_2=make_images_list.pop(0)
                    break
                else:
                    time.sleep(0.5)

            if not bloke:
                if 条件をメタデーターとして追加する:
                    pnginfo = PngImagePlugin.PngInfo()
                    info = make_image.info
                    for key, value in metadata.items():
                        pnginfo.add_text(key, str(value))
                    make_image.save(path_new, pnginfo=pnginfo)
                else:
                    make_image.save(path_new)
                if 画像情報を表示:
                    print(text_1.format(T+1),end="")
                    run_html_js(path_new,"保存先のパス: ")
                if 画像を表示:
                    print()
                    display(make_image)
                if 画像情報を表示:
                    print(text_2)
            else:
                print("ブロックされました\n")

    def grid_task(self):
        try:
            y=y
        except:
            y=1
        while True:
            grid_test_path=os.path.join(self.Grid_save_path,f"Grid_imgs-{y}.png")
            if os.path.exists(grid_test_path):
                y+=1
            else:
                grid_path=grid_test_path
                break
        if num_imgs>1:
            grid = image_grid(self.grid_imgs,cols=self.num_grid)
            if grid is not None:
                grid.save(grid_path)
                grid_img = Image.open(grid_path)
                if グリッド画像を表示:
                    run_html_js(grid_path,"グリッド画像のパス: ")
                    #print(f"\033[34mgrid_save_path: \033[32m{grid_path}\033[0m")
                    display(grid_img)
        print("\033[38;2;135;206;235m1枚あたりの生成時間の平均: {:.2f}s\033[0m".format(self.generate_time_all_after))

    def generate_video(self,seed,generator,image_save_path,sd_step,model_name,file_name,main_pipe):
        T=1
        chunk_size=4
        path_new,file_new=path_token_video(self.Prompt_4,seed,model_name,guidance_scale,file_name,image_save_path)
        now = datetime.datetime.now()
        date_str = now.strftime("%Y-%m-%d_UTC-%H:%M:%S")
        path = os.path.join(image_save_path, file_name)
        generate_start_time = time.time()
        generator = torch.Generator(device)
        generator.manual_seed(seed)
        result = []
        chunk_ids = np.arange(0, video_length, chunk_size - 1)
        for T in range(len(chunk_ids)):
            print(f"生成プロセス {T + 1} / {len(chunk_ids)}")
            ch_start = chunk_ids[T]
            ch_end = video_length if T == len(chunk_ids) - 1 else chunk_ids[T + 1]
            frame_ids = [0] + list(range(ch_start, ch_end))
            output = main_pipe(prompt=self.Prompt_4, video_length=len(frame_ids),num_inference_steps=sd_step,t0=(sd_step-5),t1=(sd_step-3) ,negative_prompt=negative_prompt2, guidance_scale=guidance_scale,height=height,width=width,generator=generator, frame_ids=frame_ids)
            result.append(output.images[1:])
        result = np.concatenate(result)
        result = [(r * 255).astype("uint8") for r in result]
        imageio.mimsave(path_new, result, fps=int(video_fps))
        generate_end_time=time.time()
        generate_time=generate_end_time-generate_start_time
        print(f"\033[34m動画生成が完了しました  {generate_time}s")
        if 画像情報を表示:
            print(f"\033[34mseed値:\033[32m {seed}")
            print(f"\033[34mファイルの名前:( \033[32m{file_new})\033[0m")
            run_html_js(path_new,"動画ファイルのパス: ")
        play_mp4(path_new)

    def main_2(self,Prompt_list,make_images_list,seed,generator,num_grid,image_save_path,sd_step,model_name,file_name,main_pipe):
        if pipeline_class_type=="_txt2img":
            self.generate_images(Prompt_list,make_images_list,seed,generator,image_save_path,sd_step,model_name,file_name,main_pipe)
        else:
            if not self.img_dir:
                self.img_path=self.img_path
                self.init_image=img_set(self.img_path)
                self.generate_images(Prompt_list,make_images_list,seed,generator,image_save_path,sd_step,model_name,file_name,main_pipe)
            else:
                for img_path in self.img_dict:

                    self.init_image=img_set(img_path)
                    self.generate_images(Prompt_list,make_images_list,seed,generator,image_save_path,sd_step,model_name,file_name,main_pipe)


    def main_t(self,seed,generator,num_grid,image_save_path,sd_step,model_name,file_name,main_pipe):
        if __name__ == '__main__':
            self.GPU_P = threading.Thread(target=self.main_2, args=(Prompt_list,make_images_list,seed,generator,num_grid,image_save_path,sd_step,model_name,file_name,main_pipe))
            self.GPU_last = threading.Thread(target=self.last_task,args=(make_images_list,))
            self.GPU_P.start()
            self.GPU_last.start()
            self.GPU_last.join()
        self.generate_time_all_after=self.generate_time_all/self.num_imgs
        self.grid_task()




generate_cls=generate_class(main_pipe,Prompt_list,make_images_list,Grid_save_path,img_dir,img_path,img_dict,seed,generator,num_grid,Prompt_4,image_save_path,sd_step,model_path,file_name)
if pipeline_class_type=="_txt2video":
  generate_cls.generate_video(seed,generator,image_save_path,sd_step,model_path,file_name,main_pipe)
else:
  generate_cls.main_t(seed,generator,num_grid,image_save_path,sd_step,model_path,file_name,main_pipe)

#追加オプション

In [ ]:
#@title #追加設定 {display-mode: "form"}

# @markdown >スケジューラーがCustomEulerA or CustomEuler の場合のみ
momentum = 1 # @param {type:"slider", min:0.8, max:1, step:0.05}
momentum_hist = -0.1 # @param {type:"slider", min:-1, max:1, step:0.1}
history_d = "rand_init" # @param ["rand_new", "rand_init"]


# @markdown >img2img、Inpaintのみ
strength = 1 # @param {type:"slider", min:0, max:1, step:0.1}
入力する画像 = "/content/IMG_2576.jpeg" #@param {type:"string"}


#@markdown >txt2video のみ
video_length= 8  # @param {type:"number"}
video_fps = 18 # @param {type:"slider", min:4, max:20, step:2}


#@markdown >safeのみ

safety_level = "WEAK" # @param ["MAX","STRONG","MEDIUM","WEAK"]


In [ ]:
import importlib
import inspect


def get_call_method(class_name:str,method_name:str='__call__') ->list:
    call_method = getattr(getattr(diffusers, class_name),method_name)
    parameters = inspect.signature(call_method).parameters
    arg_names = []
    for param in parameters.values():
        arg_names.append(param.name)
    return arg_names
print(get_call_method("StableDiffusionPipeline","__init__"))

['self', 'vae', 'text_encoder', 'tokenizer', 'unet', 'scheduler', 'safety_checker', 'feature_extractor', 'image_encoder', 'requires_safety_checker']


In [ ]:
import importlib
import inspect

module_name = "diffusers"
class_name = "StableDiffusionPipeline"

try:
    module = importlib.import_module(module_name)
    if hasattr(module, class_name):
        class_object = getattr(module, class_name)
        call_method = getattr(class_object, '__call__')
        signature = inspect.signature(call_method)
        parameters = signature.parameters

        arg_names = []
        for param in parameters.values():
            if param.default != inspect.Parameter.empty:
                arg_names.append(f"{param.name}={param.default}")
            else:
                arg_names.append(param.name)

        print(f"{class_name} の __call__ メソッドの引数リスト: {arg_names}")
    else:
        print(f"{class_name} クラスが見つかりません")
except ImportError:
    print(f"{module_name} モジュールが見つかりません")

StableDiffusionPipeline の __call__ メソッドの引数リスト: ['self', 'prompt=None', 'height=None', 'width=None', 'num_inference_steps=50', 'timesteps=None', 'guidance_scale=7.5', 'negative_prompt=None', 'num_images_per_prompt=1', 'eta=0.0', 'generator=None', 'latents=None', 'prompt_embeds=None', 'negative_prompt_embeds=None', 'ip_adapter_image=None', 'ip_adapter_image_embeds=None', 'output_type=pil', 'return_dict=True', 'cross_attention_kwargs=None', 'guidance_rescale=0.0', 'clip_skip=None', 'callback_on_step_end=None', "callback_on_step_end_tensor_inputs=['latents']", 'kwargs']


#サブ機能

In [ ]:
#@title メモリの初期化(Beta) {display-mode: "form"}

import gc
import torch

variable_names = ["base_pipe","txt2img_pipe","img2img_pipe","txt2video_pipe","Inpaint_pipe","safe_pipe", "tokenizer"]

for name in variable_names:
    if name in globals():
      del globals()[name]
    if name in locals():
      del locals()[name]

torch.cuda.empty_cache()
gc.collect()

3701

In [ ]:
#@title 画像のメタデータを見る  {display-mode: "form"}
from PIL import Image, PngImagePlugin
import os
image_path = "/content/Generated_images/Images/GIMG-68.png" #@param {type:"string"}

try:
  output_info = Image.open(image_path).info
except:
  raise FileNotFoundError("画像を読み込めませんでした")

if image_path and os.path.isfile(image_path):
    output_info = Image.open(image_path).info

    for key,info in output_info.items():
        print(f"{key} : {info}\n")






In [ ]:
#@title txt2audio
prompt = "Techno-style  music" #@param {type:"string"}
negative_prompt = "Low quality." #@param {type:"string"}
audio_length_in_s = 30 #@param {type:"number"}
num_inference_steps = 500 #@param {type:"number"}
seed = -1 #@param {type:"number"}

prompt+=",masterpiece"
import os
import scipy
import torch
from diffusers import AudioLDM2Pipeline
from IPython.display import Audio
import random



repo_id = "cvssp/audioldm2"
if "pipe" not in locals() and "pipe" not in globals():
    pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch.float16).to("cuda")
if "numa" in globals():
    numa+=1
else:
    numa=1
if seed is None or seed==-1:
    seed = random.randint(1,1000000)
generator = torch.Generator("cuda").manual_seed(seed)
audio = pipe(prompt,
             negative_prompt=negative_prompt,
             num_inference_steps=num_inference_steps,
             audio_length_in_s=audio_length_in_s,
             num_waveforms_per_prompt=1,
             generator=generator,
             ).audios


os.makedirs("/content/audio",exist_ok=True)
save_path=f"/content/audio/audio_{numa}_{seed}.wav"


scipy.io.wavfile.write(save_path, rate=16000, data=audio[0])
print(f"\033[38;2;0;255;255mseed: {seed}")
print(f"save_path: {save_path}\n\033[0m")
Audio(save_path)


In [ ]:
#@title Youtube_download {display-mode: "form"}
import os
from google.colab import files
import shutil

try:
    import pytube
except ModuleNotFoundError:
    !pip install -q pytube

from pytube import (Playlist, YouTube)
from requests import HTTPError
Youtube_path = "" # @param {type:"string"}

ローカルにダウンロードする = False # @param {type:"boolean"}
def Youtube_download(Youtube_path,local_download):
    Not_download_list=[]
    mkdir_list=["/content/Youtube/File","/content/Youtube/List"]
    for mk_path in mkdir_list:
        os.makedirs(mk_path,exist_ok=True)
    os.chdir("/content/Youtube")
    if Youtube_path:
        if Youtube_path.startswith("https://www.youtube.com/playlist?list="):
            try:
                nu+=1
            except:
                nu=1
            list_path=f"/content/Youtube/Yotube_list_{nu}"
            os.makedirs(list_path,exist_ok=True)
            os.chdir(list_path)
            Y_list = Playlist(Youtube_path)
            for video, path in zip(Y_list.videos, Y_list.video_urls[:3]):
                try:
                    video.streams.first().download()
                except:
                    Not_download_list.append(path)
            if Not_download_list:
                print("ダウンロードに失敗したパス")
                for ndl in Not_download_list:
                    print(ndl)
            if local_download:
                zip_path=f"/content/Youtube/zip-No_{zip_number}.zip"
                shutil.make_archive(zip_path, "zip", list_path)
                files.download(zip_path)

        elif Youtube_path.startswith("https://www.youtube.com/watch?v="):
            os.chdir("/content/Youtube/File")
            try:
                yt=YouTube(Youtube_path)
                yt.streams.first().download()
                yt_save_path=os.path.join("/content/Youtube/File" , (yt.title+".mp4"))
            except:
                raise HTTPError("URLが無効です")
            files.download(yt_save_path)
        else:
            raise HTTPError("URLが無効です")
    else:
        raise HTTPError("URLの入力をお願いします")

    os.chdir("/content")
    print("処理が終了しました")

Youtube_download(Youtube_path,ローカルにダウンロードする)





In [ ]:
#@title  グリッド画像の分割  (iamge generation){display-mode: "form"}
from PIL import Image
import os
import sys
import math

### 各定数の設定
# 縦方向分割数

# 分割元ファイルパス
input_path = "" # @param {type:"string"}
output_dir = "" # @param {type:"string"}
縦の分割回数 = 1 # @param {type:"number"}
横の分割回数 = 3 # @param {type:"number"}
#@markdown >heightとwidthは2以上の自然数の入力をお願いします
height=縦の分割回数
width=横の分割回数
base_img_path=input_path


if base_img_path=="" or  output_dir == "":
  raise TypeError("pathが未入力です")
if output_dir is not dir:
   output_dir = os.path.dirname(output_dir)

def is_positive_integer(value):
    try:
        value = int(value)  # 整数に変換
        if value > 0:  # 1以上の場合は True を返す
            return True
        else:
            return False
    except ValueError:  # ValueError が発生した場合は False を返す
        return False

if not is_positive_integer(height) or is_positive_integer(width):
  raise TypeError("heightとwidthは、2以上の自然数である必要があります。")

def ImgSplit(im,w,h):
    # 読み込んだ画像の高さと幅を指定分割数で割る
    HEIGHT = h / height
    WIDTH = w / width

    # 縦の分割枚数
    for h1 in range(height):
        # 横の分割枚数
        for w1 in range(width):
            w2 = w1 * WIDTH
            h2 = h1 * HEIGHT
            yield im.crop((w2, h2, WIDTH + w2, HEIGHT + h2))

#if __name__ == '__main__': このセルが直接実行された場合
    # 画像の読み込み
im = Image.open(base_img_path)
w = im.size[0]
h = im.size[1]
length = math.log10(height * width) + 1
os.makedirs(output_dir, exist_ok=True)
for number, ig in enumerate(ImgSplit(im,w,h), 1):
    # 出力
    ig.save(output_dir + "/" + str(number).zfill(int(length)) + ".PNG", "PNG")
print(f"\033[34m分割が完了しました: {output_dir}\033[0m")

In [ ]:
#@title  単語が含まれているファイルを移動{display-mode: "form"}
import os
import shutil

def move_files_with_keyword(source_dir, destination_dir, keyword):
    # 指定されたディレクトリ内のファイルを取得
    files = os.listdir(source_dir)

    for file in files:
        # ファイル名にキーワードが含まれているかチェック
        if keyword in file:
            # 移動元のファイルパス
            source_file = os.path.join(source_dir, file)
            # 移動先のファイルパス
            destination_file = os.path.join(destination_dir, file)

            # ファイルを移動
            shutil.move(source_file, destination_file)

# 使用例
検索対象のフォルダ = "/content/drive/MyDrive/GS"  # @param {type:"string"}
移動先 = "/content/drive/MyDrive/GS_grid"   # @param {type:"string"}
単語 = "grid_imgs" # @param {type:"string"}

source_directory=検索対象のフォルダ
destination_directory=移動先
keyword_to_search=単語
os.makedirs(destination_directory)
move_files_with_keyword(source_directory, destination_directory, keyword_to_search)

In [ ]:
#@title  ダウンロード{display-mode: "form"}
#@markdown >ファイル or ディレクトリを指定してダウンロードします。


from google.colab import files
import shutil
import os
path = "/content/Generated_images/Images/GIMG-68.png" #@param {type:"string"}

# 最後が特定の単語であるかをチェックする
if path.endswith(".png" or ".jpg"):
  try:
    files.download(path)
    print("\033[32mダウンロードが正常に完了しました")
  except:
    print("\033[31m指定された画像が見つかりませんでした\033[0m")
elif path.endswith(".mp4"):
  try:
    files.download(path)
    print("\033[32mダウンロードが正常に完了しました")
  except:
    print("\033[31m指定された動画ファイルが見つかりませんでした\033[0m")
else:
  try:
    zip_number+=1
  except:
    zip_number=1
  zip_name=f"Generated_images-No.{zip_number}"
  zip_save_dir=os.path.join("/content",zip_name)
  zippath=zip_save_dir+".zip"
  try:
    shutil.make_archive(zip_save_dir, "zip", path)
    files.download(zippath)
    print("\033[32mzipファイルへ圧縮が正常に完了しました")
    print(f'zipファイル名前は\033[34m"{zip_name}"\033[32mですご確認ください\033[0m')
  except:
    zip_number-=1
    print("\033[31m指定されたディレクトリが見つかりませんでした\033[0m")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ダウンロードが正常に完了しました


In [ ]:
#@title dir_size_counter {display-mode: "form"}
#@markdown
import os

def get_folder_size(folder_path):
    total_size = 0
    if os.path.isfile(folder_path):
        total_size+=os.path.getsize(folder_path)
    else:
        for path, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(path, file)
                total_size += os.path.getsize(file_path)

    return total_size

folder_path = "" # @param {type:"string"}
if not os.path.exists(folder_path):
    raise FileNotFoundError("pathが無効です")

size_in_bytes = get_folder_size(folder_path)


size_in_kb = size_in_bytes / 1024
size_in_mb = size_in_kb / 1024
size_in_gb = size_in_mb / 1024

print(f"Folder Size: {size_in_bytes} bytes")
print(f"Folder Size: {size_in_kb:.2f} KB")
print(f"Folder Size: {size_in_mb:.2f} MB")
print(f"Folder Size: {size_in_gb:.2f} GB")

In [ ]:
#@title 画像変換 {display-mode: "form"}
import os
from PIL import Image
import sys
input_path = "" #@param {type:"string"}
output_path="" #@param {type:"string"}
Change_mode = "png2jpg" # @param ["png2jpg", "jpg2png"]

#@markdown * png2jpg :png画像をjpg画像に変換します。


#@markdown * jpg2png :jpg画像をpng画像に変換します。

if not os.path.exists(input_path):
  #raise FileNotFoundError('\033[33m指定された "dir"または"file" が見つかりませんでした。\033[0m')
  sys.exit('"input_path"にdirまたはfileのパスを入力お願いします。')

if not input_path.endswith(".png" or ".jpg" or ".jpeg"):
  if output_path=="":
    # 保存先のディレクトリパスを作成
    save_dir_path = input_path +"-" +Change_mode
  else:
    save_dir_path=output_path
  # 保存先のディレクトリが存在しない場合は作成する
  if not os.path.exists(save_dir_path):
    os.makedirs(save_dir_path)

  # ディレクトリ内のファイルを取得
  file_list = os.listdir(input_path)

  # ファイルごとに処理を行う
  for file_name in file_list:
    # ファイルのパスを作成
    file_path = os.path.join(save_dir_path, file_name)

    # ファイルの拡張子を取得
    file_extension = os.path.splitext(file_name)[1]
    if Change_mode == "png2jpg":
    # 最後が特定の単語であるかをチェックする
      if file_extension.endswith(".png"):
        try:
          # 画像を開く
          im = Image.open(file_path)
          file_name_new = file_name.replace(".png", ".jpg")#pngをjpg
          # 保存先のファイルパスを作成
          save_file_path = os.path.join(save_dir_path, file_name_new)
          # 変換後の画像を保存する
          im.save(save_file_path,format="JPEG")
          print(f"\033[34m{save_file_path}を保存しました。\033[0m")
        except Exception as e:
          print(f"\033[31m{file_name}の変換中にエラーが発生しました: {str(e)}\033[0m")
      else:
        print(f"{file_name}はpngファイルではないため、変換をスキップしました。")
    elif Change_mode == "jpg2png":
        try:
          # 画像を開く
          im = Image.open(file_path)
          file_name_new = file_name.replace((".jpg"or".jpeg"), ".png")#jpgをpng
          # 保存先のファイルパスを作成
          save_file_path = os.path.join(save_dir_path, file_name_new)
          # 変換後の画像を保存する
          im.save(save_file_path,format="PNG")
          print(f"\033[34m{save_file_path}を保存しました。\033[0m")
        except Exception as e:
          print(f"\033[31m{file_path}の変換中にエラーが発生しました: {str(e)}\033[0m")
else:
  if Change_mode =="png2jpg":
    if not output_path=="":
      basename_new = input_path.replace(".png", ".jpg")#jpgをpng
      save_file_path=os.path.join(output_path, basename_new)
      im = Image.open(input_path)
      im.save(save_file_path,format="JPEG")
    else:
      im = Image.open(input_path)
      save_file_path = input_path.replace(".png", ".jpg")
      im.save(save_file_path,format="JPEG")
    print(f'\033[34m"{save_file_path}"に保存しました。\033[0m')
  elif Change_mode == "jpg2png":
    if not output_path=="":
      basename_new = input_path.replace((".jpg"or".jpeg"), ".png")#jpgをpng
      save_file_path=os.path.join(output_path, basename_new)
      im = Image.open(input_path)
      im.save(save_file_path,format="PNG")
    else:
      im = Image.open(input_path)
      save_file_path = input_path.replace((".jpg"or".jpeg"), ".png")
      im.save(save_file_path,format="PNG")
    print(f'\033[34m"{save_file_path}"に保存しました。\033[0m')

In [ ]:
#@title mp4を再生  {display-mode: "form"}
MP4_path= "" #@param {type:"string"}
# mp4動画の再生
import os
from IPython.display import HTML
from base64 import b64encode
try:
  mp4 = open(MP4_path, 'rb').read()
except:
  raise FileNotFoundError("指定されたmp4が見つかりませんでした")
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title  文字列トークンカウンター{display-mode: "form"}
import pprint
from transformers import AutoTokenizer

prompt = ""  #@param {type:"string"}
try:
  tokens = tokenizer.tokenize(prompt)
except:
  text_model_id = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
  tokenizer = AutoTokenizer.from_pretrained(text_model_id)
  tokens = tokenizer.tokenize(prompt)
print(len(tokens))
pprint.pprint(tokens)

In [ ]:
#@title 高画質化  {display-mode: "form"}
from PIL import Image
from io import BytesIO
from diffusers import StableDiffusionUpscalePipeline
import torch

# load model and scheduler
Prompt = "" #@param {type:"string"}
prompt="masterpiece:2.0,best quality,high quality,"+Prompt
low_res_img_path = "/content/IMG_1708.jpeg" #@param {type:"string"}
encoded_text = codecs.encode(low_res_img_path, 'utf-8')
low_res_img_path = codecs.decode(encoded_text, 'utf-8')

if "pipeline2" not in locals()  and "pipeline2" not in globals():
  model_id = "stabilityai/stable-diffusion-x4-upscaler"
  pipeline2 = StableDiffusionUpscalePipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16
  )
  pipeline2 = pipeline2.to("cuda")
if not os.path.exists(low_res_img_path):
  raise FileNotFoundError("ファイルが見つかりませんでした")
try:
  low_res_img = Image.open(low_res_img_path)
except:
  raise FileNotFoundError("画像を読み込めませんでした")
low_res_img = low_res_img.resize((128, 128))

negative_prompt="low quality:2.0"

upscaled_image = pipeline2(prompt=prompt, image=low_res_img,negative_prompt=negative_prompt).images[0]

try:
  L+=1
except:
  L=1
os.makedirs("/content/low_res_imgs",exist_ok=True)
path1=(f"/content/low_res_imgs/No{L}.png")
upscaled_image.save(path1)
print(f"画像保存パス: ({path1})")

In [ ]:
#@title 画像フォルダを削除  {display-mode: "form"}

del_path = "/content/drive/MyDrive/korect/Images" #@param {type:"string"}
if del_path =="":
  del_path="/content/Generated_images"
import shutil
if not os.path.isdir(del_path):
  raise TypeError("ディレクトリのみ削除が可能です")
#@markdown 未入力の場合 /content/Generated_imagesを削除します

YorS=input("本当に削除しますか？[yes/no]: ")
if YorS.lower() in ("yes", "y"): # 入力された文字列を小文字にして、yesやyと一致するか判定する
  try:
    shutil.rmtree(del_path)
    print("削除しました")
  except FileNotFoundError:
    print(f"\033[31m{del_path}が見つかりませんでした\033[0m")
elif YorS.lower() in ("no", "n"): # 入力された文字列を小文字にして、noやnと一致するか判定する
  print("フォルダの削除を中止しました")
else:
  print("yes/no のみの入力をお願いします。")

In [ ]:
#@title url_Download
import urllib.request , os
import datetime
def download_file(url, save_path):
  other_url,split=os.path.splitext(url)
  if save_path=="":
    save_path="/content/download"
    choice_name=input("ファイル名(拡張子無し): ")
    choice_name+=split
    test_path=os.path.join(save_path,choice_name)
    if os.path.exists(test_path):
      now=datetime.now()
      save_path=test_path+now
  save_dir=os.path.dirname(save_path)
  os.makedirs(save_dir,exist_ok=True)
  if not url.endswith(split):
    save_path+=split
  try:
    urllib.request.urlretrieve(url, save_path)
  except HTTPError:
    raise SystemError("URLからファイルを習得できませんでした")
  print(f"保存先のパス: {save_path}")

url = "" #@param {type:"string"}
save_path = "" #@param {type:"string"}

download_file(url, save_path)

In [ ]:
#@markdown >そのうち使うかもしれないclass
class fomat_class:
    def to_list(self,string):
        return_list=[]
        if type(string) is str:
            return_list.append(string)
            return return_list
        else:
            return list(string)


    def input_fomat(self,Value,zero_txt="", return_txt="",print_txt="パス",auto=False):
        if zero_txt and return_txt:
            special_state=True
            num=0
        else:
            special_state=False
            num=1

        if type(Value) is dict:
            Value_is_dict=True
            first_value_length = len(list(Value.values())[0])
            for value in my_dict.values():
                if len(value) != first_value_length:
                   raise TypeError("The number of elements per key in the dictionary must be uniform")
        else:
            Value_is_dict=False
        if auto:
            if Value_is_dict:
                return list(Value.get(list(Value.keys())[0], []))
            else:
                choice_path=Value[0]
                return self.to_list(choice_path)

        if len(Value)>=15:
            start_number="1"
            if special_state:
                start_number="0"
                print(f"\033[34m0.{zero_txt}")

            if Value_is_dict:
                sorted_with_like = sorted(Value.items(), key=lambda x: x[1][1], reverse=True)
                for i, (model_id, (model_name, like)) in enumerate(sorted_with_like, 1):
                    if i>15:
                        break
                    print(f"\033[34m{i}.{print_txt}: {model_name}, 評価: {like}")
            else:
                for i in range(1,16):#15個
                    print(f"\033[34m{i+1}.{print_txt}: {Value[i]}\033[0m")


            print(f"\033[34m16.上記の{print_txt}以外 (全ての候補を表示します)\033[0m\n")
            while True:
                choice = input(f"{print_txt}の選択({start_number}~16): ")
                try:
                    choice=int(choice)
                except:
                    print("\033[33m自然数のみ有効です\033[34m")
                    continue
                if special_state and choice==0:
                    print("\033[0m",end="")
                    return self.to_list(return_txt)

                if choice==16: #other_file
                    break
                elif 1<=choice<=15:
                    if Value_is_dict:
                        choice_path = Value.get(list(Value.keys())[choice - 1], [])
                    else:
                        choice_path=Value[choice-1]
                    print("\033[0m",end="")
                    return self.to_list(choice_path)
                else:
                    print(f"\033[33m{num}~16 までの数字の入力をお願いします\033[34m")

        start_number="1"
        if special_state:
            start_number="0"
            print(f"\033[34m0.{zero_txt}")
        for i, file_name in enumerate(Value, 1):
            print(f"\033[34m{i}.{print_txt}: {file_name}")
        while True:
            choice = input(f"使用する{print_txt}の選択({start_number}~{len(Value)}): ")
            try:
                choice=int(choice)
            except:
                print("\033[33m自然数のみ有効です\033[34m")
                continue
            print("\033[0m",end="")
            if special_state and choice==0:
                return self.to_list(return_txt)
            elif 1<=choice<=len(Value):
                if Value_is_dict:
                    choice_path = Value.get(list(Value.keys())[choice - 1], [])
                    return self.to_list(choice_path)
                else:
                    choice_path=Value[choice-1]
                    return self.to_list(choice_path)
            else:
                print(f"\033[33m{num}~{len(Value)} までの数字の入力をお願いします\033[34m")
format=format_class

#学習

In [ ]:
#@title モデルを学習 {display-mode: "form"}

学習で使用する単語 = "Cat ears girl" # @param {type:"string"}
#概念を説明する単語 = "" # @param {type:"string"}
学習ステップ = 20   # @param {type:"number"}

入力する画像フォルダ = "/content/drive/MyDrive/GS" # @param {type:"string"}
出力するフォルダ = "/content/drive/MyDrive/make_model_2" # @param {type:"string"}
#概念のタイプ = "style" # @param ["style","object"]
モデル名前_or_パス = "/content/drive/MyDrive/loliDiffusion_SFW_.safetensors" # @param ["runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1"] {allow-input: true}
#出力ファイル名 = "" # @param {type:"string"}


#@markdown ---

終了したらランタイムを切断 = False # @param {type:"boolean"}
#@markdown sd1.5系列とsd2.1系列では埋め込みに互換性がありません。

driveに接続 = True # @param {type:"boolean"}

# @markdown >用語の説明
# @markdown * placeholder_token : 学習内で、使用するトークンです。新しい概念を入力します。
# @markdown * initializer_token : 新しい概念が何であるかを要約する単語です。
import os
from google.colab import drive
drive_cn=driveに接続
if driveに接続:
  if not drive._os.path.ismount('/content/drive'):
    try:
      drive.mount('/content/drive')
    except:
      print("ドライブの接続に失敗しました。")
      drive_cn=False



if  drive_cn==False and "/content/drive/MyDrive" in 出力するフォルダ:
  raise TypeError("Googleドライブに接続されていないため、ドライブに保存できません。")

if not os.path.isdir(入力する画像フォルダ):
  raise FileNotFoundError("画像フォルダが存在しません")

if 出力するフォルダ is None:
  raise FileNotFoundError("結果を出力するフォルダを入力してください")



os.makedirs(出力するフォルダ,exist_ok=True)
try:
  import diffusers,transformers
except:
  !pip install transformers diffusers -q
  import transformers,diffusers

if not os.path.exists("/content/script/diffusers"):
  %cd /content/script
  !git clone https://github.com/huggingface/diffusers
  %cd diffusers

try:
  import accelerate,transformers,ftfy
except:
  !pip install -q accelerate>=0.16.0 transformers>=4.25.1 ftfy Jinja2 bitsandbytes
  import bitsandbytes,accelerate,transformers,ftfy
import accelerate,torchvision,transformers,ftfy ,diffusers,bitsandbytes

size=512
if モデル名前_or_パス=="runwayml/stable-diffusion-v1-5":
  size=512
#else:
#  size=768

#exportは使用不可
%env pretrained_model_name_or_path=$モデル名前_or_パス
%env train_data_dir=$入力する画像フォルダ
%env placeholder_token=$学習で使用する単語
%env resolution=$size
%env max_train_steps=$学習ステップ
%env output_dir=$出力するフォルダ


%cd /content/script/diffusers/examples/textual_inversion

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$pretrained_model_name_or_path  \
  --instance_data_dir=$train_data_dir \
  --output_dir=$output_dir \
  --instance_prompt=$placeholder_token \
  --resolution=$resolution \
  --train_batch_size=1 \
  --learning_rate=1 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$max_train_steps \

base=os.path.join(出力するフォルダ,"learned_embeds.safetensors")
if not 出力ファイル名=="":
  after_name=出力ファイル名+".safetensors"
  after=os.path.join(出力するフォルダ,after_name)
  if os.path.exists(base):
    os.rename(base,after)
  else:
    print("埋め込みファイルが見つかりませんでした。")
else:
  after=base

print(f"""\033[34m
学習が終了しました
path: {出力するフォルダ}\033[0m
""")

if 終了したらランタイムを切断:
  from google.colab import runtime
  print("ランタイムを切断中...")
  runtime.unassign()

In [ ]:
#@title 埋め込みを学習 {display-mode: "form"}

学習で使用するトークン = "flans" # @param {type:"string"}
概念を説明する単語 = "flandre" # @param {type:"string"}
学習ステップ = 50   # @param {type:"number"}

入力する画像フォルダ = "/content/drive/MyDrive/GS" # @param {type:"string"}
出力するフォルダ = "/content/drive/MyDrive/textjual" # @param {type:"string"}
概念のタイプ = "style" # @param ["style","object"]
モデル名前_or_パス = "runwayml/stable-diffusion-v1-5" # @param ["runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1"] {allow-input: true}

出力ファイル名 = "Flans" # @param {type:"string"}


#@markdown ---

終了したらランタイムを切断 = False # @param {type:"boolean"}
#@markdown sd1.5系列とsd2.1系列では埋め込みに互換性がありません。

driveに接続 = True # @param {type:"boolean"}

# @markdown >用語の説明
# @markdown * placeholder_token : 学習内で、使用するトークンです。新しい概念を入力します。
# @markdown * initializer_token : 新しい概念が何であるかを要約する単語です。
import os
from google.colab import drive
drive_cn=driveに接続
if driveに接続:
  if not drive._os.path.ismount('/content/drive'):
    try:
      drive.mount('/content/drive')
    except:
      print("ドライブの接続に失敗しました。")
      drive_cn=False



if  drive_cn==False and "/content/drive/MyDrive" in 出力するフォルダ:
  raise TypeError("Googleドライブに接続されていないため、ドライブに保存できません。")

if not os.path.isdir(入力する画像フォルダ):
  raise FileNotFoundError("画像フォルダが存在しません")

if 出力するフォルダ is None:
  raise FileNotFoundError("結果を出力するフォルダを入力してください")



os.makedirs(出力するフォルダ,exist_ok=True)


try:
  import diffusers,transformers
except:
  !pip install transformers diffusers -q
  import transformers,diffusers

if not os.path.exists("/content/script/diffusers"):
  %cd /content/script
  !git clone https://github.com/huggingface/diffusers




if "setup_txt" not in locals():
  %cd /content/script/diffusers/examples/textual_inversion
  !pip install -r ./requirements.txt -q
  setup_txt=True

import accelerate,torchvision,transformers,ftfy ,diffusers


%cd /content/diffusers/examples/textual_inversion

size=512
if モデル名前_or_パス=="runwayml/stable-diffusion-v1-5":
  size=512
else:
  size=768

#exportは使用不可
%env pretrained_model_name_or_path=$モデル名前_or_パス
%env train_data_dir=$入力する画像フォルダ
%env learnable_property=$概念のタイプ
%env placeholder_token=$学習で使用するトークン
%env initializer_token=$概念を説明する単語
%env resolution=$size
%env max_train_steps=$学習ステップ
%env output_dir=$出力するフォルダ

#--learning_rate=5  \
#--mixed_precision="fp16" \
#learning_rateは整数
!accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path=$pretrained_model_name_or_path \
  --train_data_dir=$train_data_dir \
  --learnable_property=$learnable_property \
  --placeholder_token=$placeholder_token \
  --initializer_token=$initializer_token \
  --resolution=$resolution \
  --train_batch_size=4 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=$max_train_steps \
  --lr_scheduler="constant" \
  --learning_rate=5  \
  --lr_warmup_steps=0 \
  --output_dir=$output_dir

base=os.path.join(出力するフォルダ,"learned_embeds.safetensors")
if not 出力ファイル名=="":
  after_name=出力ファイル名+".safetensors"
  after=os.path.join(出力するフォルダ,after_name)
  if os.path.exists(base):
    os.rename(base,after)
  else:
    print("埋め込みファイルが見つかりませんでした。")
else:
  after=base

print(f"""\033[34m
学習が終了しました
path: {after}\033[0m
""")

if 終了したらランタイムを切断:
  from google.colab import runtime
  print("ランタイムを切断中...")
  runtime.unassign()

#説明書

In [ ]:
#@title 埋め込みを適用 {display-mode: "form"}


# @markdown * Repo_id_or_path : hugface_idまたはfile_pathの入力をお願いします。

# @markdown * token : 埋め込みを呼び出す場合に"Prompt"または"N_prompt"のいずれかに入力お願いします。

# @markdown * weight_name : Repo_idの場合にファイル名前を指定してください

# @markdown  ーstep.3の終了後に使用可能です





In [ ]:
#@title プロンプトの特殊トークン {display-mode: "form"}

# @markdown >ランダムワード
# @markdown
# @markdown * {cat,dog},cute = "cat , cute" / "dog , cute"
# @markdown * color,{blue,red,green} = "color , blue" / "color , red" / "color , green"

#@markdown >''=文字のスイッチ

#@markdown * man,{boy,girl, ' '} = "man" / "man,boy" / "man,girl"

#@markdown >file_nameの特殊トークンについて

#@markdown ファイル名にパラメータなどの値を入力できます。

#@markdown * {prompt} : Prompt

#@markdown * {seed} : seed値

#@markdown * {model_name} : model_name

#@markdown * {g_scale} : guidance_scale

#@markdown * {time} : 現在時刻

#@markdown * {number} : 生成した回数

In [ ]:
#@title "自動で条件を決めて生成" の詳細 {display-mode: "form"}


#@markdown  * 詳細設定を推奨の値に設定

#@markdown   * 次の機能をオンにします
#@markdown   * 画面に表示
#@markdown   * 画像の質を上げるプロントを追加する
#@markdown   * プロンプトアシストを使う ( MagicPrompt )
#@markdown   * 推奨するネガティブプロントを使用
#@markdown   * 条件をメタデーターとして追加する



In [ ]:
#@title プロンプトアシスタントの詳細 {display-mode: "form"}

# @markdown アニメ調の画像に適したアシスタントは " anime-anything-promptgen-v2 "

# @markdown 多目的のアシスタントは " MagicPrompt-Stable-Diffusion "

# @markdown 使用しない場合は " None " の選択をお願いします

In [ ]:
#@title 用語の説明 {display-mode: "form"}

# @markdown * txt2img  : テキストから画像

# @markdown * img2img : 画像から画像

# @markdown * Inpaint : 書いた画像に色をつける

# @markdown * txt2video : テキストから動画

# @markdown * safe : より安定した安全な画像の生成(txt2img)

# @markdown * seed ("-1"以上) / seed値を指定します。0の場合ランダムな数字を割り当てます。

# @markdown * guidance_scale (5≦15 推奨"7.5") / promptの強さの値です。強すぎるとノイズが発生する一方、弱すぎると絵が崩壊します。

# @markdown * history_d ("rand_new", "rand_init" )  推奨:"rand_new

# @markdown * moment(0.1≦1.0 ) 推奨"0.8"

# @markdown * momentum_hist(-1.0≦1.0)  推奨"0.2"

#@markdown * 拡散ステップ(1≦1000 推奨"50") / 計算をする回数を指定します。回数を減らすほど生成速度が速くなります

# @markdown * 縦・横の大きさ ( 推奨"512" ) / 大きくすればするほど生成速度が遅くなります(img2img、Inpaint**以外のみ**)

#@markdown  * safety_level(強さ) ： WEAK＜MEDIUM＜STRONG＜MAX

#@markdown  * 条件を統一 : プロンプトアシスタントを使用するとき、最初の画像のプロンプトを繰り返し使用します



In [ ]:
#@title デフォルト(値が未入力の場合) {display-mode: "form"}

#@markdown >保存する先のパス
#@markdown * デフォルトでは /content/Generated_images に保存されます。なければ作るようになっています。
#@markdown * ドライブに保存する場合 /content/drive/MyDriveを最初につけてください

# @markdown >file_name
#@markdown * デフォルトは "GIMG-{number}" です



In [ ]:
#@title 追記 {display-mode: "form"}

# @markdown 2024.1.27日時点でmoment付きEuler または EulerA において、history_d を "rand_init" にすると、1枚目が崩壊するバグがあります。
# @markdown なお、通常のEuler系のサンプラーでは起きません

#@markdown 条件: moment:0.8、moment_hist:0.6





#Readme

>重要な注意点 :
* 商用利用はご遠慮ください。
* 画像生成によって起こった問題について、当方は一切責任を負いません。

>免責事項:
*  使用にあたっては、自己責任でお願いします。
*  本ノートブックは予告なく変更・非公開・削除する可能性があります。
*  利用規約は予告なく変更する場合があります。
*  このモデルは、趣味で作成したものであり、商用利用などは意図していません。
*  使用にあたって発生した通信量、電気料金など金銭に関わるものの負担は追い兼ねます

*  本プロジェクトを利用することにより生じた一切の問題について、当方は一切責任を負いません。

ー本プロジェクトとは、本画像生成ノートブックや、githubのページなどをさします
___
>本プロジェクトの説明
* プログラミングの個人的な学習
* Stable Diffusionをベースにした画像生成ノートブックです。


---

謝辞

本画像生成ノートブックの作成にあたり、オープンソースのリソースやフリーのツールを使用させていただきました。個人的な利用でしたが、これらのリソースやツールがあったからこそ、本プロジェクトを実現することができました。
この場を借りて、オープンソースのコミュニティや、フリーのツールを提供してくださった方々に感謝の意を表します。素晴らしいツールや技術を提供してくださり、本プロジェクトを支援してくださったことに心から感謝いたします。